In [96]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO

# from simple_salesforce import Salesforce
# import shutil
# import datetime
# from datetime import date
import json, re

# from pathlib import Path
# import pandas as pd
# import numpy as np
# from io import StringIO
import shutil
import datetime
from datetime import datetime, date

# import re
# import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()


""" sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
) """

' sf = Salesforce(\n    username="jmartinez@careassistance.com",\n    password="Jcmm01041983*",\n    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",\n) '

Empresas Programas por empresa y empresas desde account


In [97]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [98]:
# para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
# para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

CITA DE SERVICIO


In [99]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from ServiceAppointment WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.ServiceAppointment.update(fake_dict, batch_size=10000, use_serial=True)
Cita_Servicio.head()

Cita_Servicio.shape

100%|██████████| 1/1 [00:06<00:00,  6.11s/it]


(23, 1)

In [100]:
Cita_Servicio = "SELECT Id,Nombre_Tipo_de_Cita__c,AppointmentNumber,Status, Interacci_n__c,CreatedBy.Name,Clinica__c,SchedStartTime,EarliestStartTime,AccountId,EmpresaContactoConvenio__c from ServiceAppointment WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
CitaDeServicio = sf.smart_query(Cita_Servicio)
CitaDeServicio["Orientación"] = CitaDeServicio.Interacci_n__c.str.contains("Ingreso")

CitaDeServicio["Realizado"] = CitaDeServicio["Status"].str.replace("Confirmado", "SI")
CitaDeServicio["Realizado"] = CitaDeServicio["Realizado"].str.replace(
    "No Confirmado", "NO"
)
CitaDeServicio["Realizado"] = CitaDeServicio["Realizado"].str.replace("Cancelado", "NO")
CitaDeServicio["Realizado"] = CitaDeServicio["Realizado"].str.replace(
    "No realizado", "NO"
)
CitaDeServicio["Realizado"] = CitaDeServicio["Realizado"].str.replace(
    "Programado", "NO"
)
CitaDeServicio["Realizado"] = CitaDeServicio["Realizado"].str.replace(
    "Reagendado", "SI"
)
CitaDeServicio["Realizado"] = CitaDeServicio["Realizado"].str.replace("Realizado", "SI")

100%|██████████| 1/1 [00:09<00:00,  9.17s/it]


Cita agendamiento


In [101]:
CitaDeServicioA = pd.DataFrame()

CitaDeServicioA["Id"] = CitaDeServicio["Id"]
CitaDeServicioA["Ficha"] = CitaDeServicio["AppointmentNumber"]
CitaDeServicioA["Fecha creación"] = CitaDeServicio["EarliestStartTime"]
CitaDeServicioA["Canal"] = CitaDeServicio["Nombre_Tipo_de_Cita__c"]
CitaDeServicioA["Realizado"] = CitaDeServicio["Status"]
CitaDeServicioA["Tipo de atención"] = "Agendamiento"
CitaDeServicioA["Profesional"] = np.where(
    CitaDeServicio["CreatedBy.Name"] == "No - Reply", "App", "Profesional"
)
CitaDeServicioA["Clínica"] = CitaDeServicio["Clinica__c"]
CitaDeServicioA["Fecha agendamiento"] = CitaDeServicio["SchedStartTime"]




CitaDeServicioO = pd.DataFrame()
CitaDeServicioO = CitaDeServicio[(CitaDeServicio["Orientación"] == True)]
# CitaDeServicioO = Pacientes_con_id[(Pacientes_con_id["RutEstaoNo"] == False)]

# CitaDeServicioO["Id"] = CitaDeServicio["Id"]
CitaDeServicioO["Ficha"] = CitaDeServicioO["AppointmentNumber"]
CitaDeServicioO["Fecha creación"] = CitaDeServicioO["EarliestStartTime"]
CitaDeServicioO["Canal"] = np.where(
    CitaDeServicioO["CreatedBy.Name"] == "No - Reply", "Aplicación", "Llamada"
)
CitaDeServicioO["Realizado"] = "SI"
CitaDeServicioO["Tipo de atención"] = np.where(
    CitaDeServicioO["CreatedBy.Name"] == "No - Reply", "Aplicación", "Llamada Ondemand"
)
CitaDeServicioO["Profesional"] = np.where(
    CitaDeServicioO["CreatedBy.Name"] == "No - Reply", "App", "Profesional"
)
CitaDeServicioO["Fecha agendamiento"] = CitaDeServicioO["SchedStartTime"]
CitaDeServicioO["Clínica"] = "Orientación Médica"
CitaDeServicioO = CitaDeServicioO[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        "Fecha agendamiento",
    ]
]

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5672\3006692268.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CitaDeServicioO["Ficha"] = CitaDeServicioO["AppointmentNumber"]
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5672\3006692268.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CitaDeServicioO["Fecha creación"] = CitaDeServicioO["EarliestStartTime"]
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5672\3006692268.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [102]:
CitaDeServicioA.head(5)

,Id,Ficha,Fecha creación,Canal,Realizado,Tipo de atención,Profesional,Clínica,Fecha agendamiento
0,08pRN000000NhHBYA0,SA-62157,1711933783000,Videollamada,Realizado,Agendamiento,App,Consulta Nutricional,1712404800000
1,08pRN000000NhInYAK,SA-62158,1711933881000,Videollamada,Realizado,Agendamiento,App,Consulta Deportiva,1712408400000
2,08pRN000000NhXJYA0,SA-62159,1711938336000,Teléfono,No realizado,Agendamiento,App,Consulta Nutricional,1711975500000
3,08pRN000000NijVYAS,SA-62162,1711972639000,Videollamada,Realizado,Agendamiento,Profesional,Consulta Psicológica,1712787000000
4,08pRN000000Nil7YAC,SA-62163,1711973400000,Videollamada,Cancelado,Agendamiento,App,Veterinaria,1712581200000


In [103]:
CitaDeServicioO.head(5)

,Id,Ficha,Fecha creación,Canal,Realizado,Tipo de atención,Profesional,Clínica,Fecha agendamiento
0,08pRN000000NhHBYA0,SA-62157,1711933783000,Aplicación,SI,Aplicación,App,Orientación Médica,1712404800000
1,08pRN000000NhInYAK,SA-62158,1711933881000,Aplicación,SI,Aplicación,App,Orientación Médica,1712408400000
2,08pRN000000NhXJYA0,SA-62159,1711938336000,Aplicación,SI,Aplicación,App,Orientación Médica,1711975500000
4,08pRN000000Nil7YAC,SA-62163,1711973400000,Aplicación,SI,Aplicación,App,Orientación Médica,1712581200000
5,08pRN000000NimjYAC,SA-62164,1711973426000,Aplicación,SI,Aplicación,App,Orientación Médica,1712581200000


Paciente Cronico

In [104]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_Seg_Paciente_Cr_nico__c.update(fake_dict, batch_size=10000, use_serial=True)
Cita_Servicio.head()

Cita_Servicio.shape

query_Cronico = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Cronico = sf.smart_query(query_Cronico)
dt_Cronico["Clínica"] = "Paciente Crónico"
dt_Cronico["Profesional"] = np.where(
    dt_Cronico["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Cronico = dt_Cronico.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Cronico = dt_Cronico[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Cronico.shape

  0%|          | 0/1 [00:00<?, ?it/s]

Deporte

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_Deporte__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_Deporte__c.update(fake_dict, batch_size=10000, use_serial=True)
Cita_Servicio.head()

Cita_Servicio.shape

query_Deporte = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Cl_nica_Deporte__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Deporte = sf.smart_query(query_Deporte)
dt_Deporte["Clínica"] = "Deporte"
dt_Deporte["Profesional"] = np.where(
    dt_Deporte["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Deporte = dt_Deporte.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Deporte = dt_Deporte[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Deporte.shape

Nutricion

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Clinica_Nutrici_n__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Clinica_Nutrici_n__c.update(fake_dict, batch_size=10000, use_serial=True)
Cita_Servicio.head()

Cita_Servicio.shape

query_nutricion = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Clinica_Nutrici_n__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Nutricion = sf.smart_query(query_nutricion)
dt_Nutricion["Clínica"] = "Nutrición"
dt_Nutricion["Profesional"] = np.where(
    dt_Nutricion["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Nutricion = dt_Nutricion.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Nutricion = dt_Nutricion[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Nutricion.shape

APP SEX

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_ApsSex__c  WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_ApsSex__c.update(fake_dict, batch_size=10000, use_serial=True)

query_appsex = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Cl_nica_ApsSex__c  WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_AppSex = sf.smart_query(query_appsex)
dt_AppSex["Clínica"] = "AppSex"
dt_AppSex["Profesional"] = np.where(
    dt_AppSex["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_AppSex = dt_AppSex.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_AppSex = dt_AppSex[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_AppSex.shape

Sueño

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_del_Sue_o__c   WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_del_Sue_o__c.update(fake_dict, batch_size=10000, use_serial=True)

query_sueno = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Cl_nica_del_Sue_o__c   WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Sueno = sf.smart_query(query_sueno)
dt_Sueno["Clínica"] = "Clínica del Sueño"
dt_Sueno["Profesional"] = np.where(
    dt_Sueno["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Sueno = dt_Sueno.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Sueno = dt_Sueno[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Sueno.shape

Salus Sexual y reproductiva

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Salud_Sexual_y_Reproductiva__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Salud_Sexual_y_Reproductiva__c.update(fake_dict, batch_size=10000, use_serial=True)

query_saludSexual = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Salud_Sexual_y_Reproductiva__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_SaludSexual = sf.smart_query(query_saludSexual)
dt_SaludSexual["Clínica"] = "Salud Sexual y Reproductiva"
dt_SaludSexual["Profesional"] = np.where(
    dt_SaludSexual["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_SaludSexual = dt_SaludSexual.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_SaludSexual = dt_SaludSexual[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_SaludSexual.shape

Asistencia Social

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Asistencia_Social__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Asistencia_Social__c.update(fake_dict, batch_size=10000, use_serial=True)

query_asistencia = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Asistencia_Social__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_AsistenciaSocial = sf.smart_query(query_asistencia)
dt_AsistenciaSocial["Clínica"] = "Asistencia Social"
dt_AsistenciaSocial["Profesional"] = np.where(
    dt_AsistenciaSocial["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_AsistenciaSocial = dt_AsistenciaSocial.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_AsistenciaSocial = dt_AsistenciaSocial[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_AsistenciaSocial.shape

100%|██████████| 1/1 [00:02<00:00,  2.50s/it]


(0, 8)

Legal Familia

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Legal_Familia__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Legal_Familia__c.update(fake_dict, batch_size=10000, use_serial=True)

query_legal = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Legal_Familia__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_LegalFamilia = sf.smart_query(query_legal)
dt_LegalFamilia["Clínica"] = "Legal Familia"
dt_LegalFamilia["Profesional"] = np.where(
    dt_LegalFamilia["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_LegalFamilia = dt_LegalFamilia.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_LegalFamilia = dt_LegalFamilia[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_LegalFamilia.shape

100%|██████████| 1/1 [00:02<00:00,  2.94s/it]


(0, 8)

Salud Bucal

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Salud_Bucal__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Salud_Bucal__c.update(fake_dict, batch_size=10000, use_serial=True)

query_bucal = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Salud_Bucal__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_SaludBucal = sf.smart_query(query_bucal)
dt_SaludBucal["Clínica"] = "Salud Bucal"
dt_SaludBucal["Profesional"] = np.where(
    dt_SaludBucal["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_SaludBucal = dt_SaludBucal.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_SaludBucal = dt_SaludBucal[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_SaludBucal.shape

100%|██████████| 1/1 [00:03<00:00,  3.43s/it]


(8, 8)

Veterinaria

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_Veterinaria__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_Veterinaria__c.update(fake_dict, batch_size=10000, use_serial=True)

query_veterinaria = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Cl_nica_Veterinaria__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Veterinaria = sf.smart_query(query_veterinaria)
dt_Veterinaria["Clínica"] = "Veterinaria"
dt_Veterinaria["Profesional"] = np.where(
    dt_Veterinaria["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Veterinaria = dt_Veterinaria.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Veterinaria = dt_Veterinaria[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Veterinaria.shape

100%|██████████| 1/1 [00:02<00:00,  2.81s/it]


(52, 8)

Mis Primeros Pasos

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Mis_Primeros_Pasos__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Mis_Primeros_Pasos__c.update(fake_dict, batch_size=10000, use_serial=True)

query_primeros = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Mis_Primeros_Pasos__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_PrimerosPasos = sf.smart_query(query_primeros)
dt_PrimerosPasos["Clínica"] = "Primeros Pasos"
dt_PrimerosPasos["Profesional"] = np.where(
    dt_PrimerosPasos["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_PrimerosPasos = dt_PrimerosPasos.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_PrimerosPasos = dt_PrimerosPasos[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_PrimerosPasos.shape

100%|██████████| 1/1 [00:03<00:00,  3.56s/it]


(4, 8)

Obstetricia

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_Obstreticia__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_Obstreticia__c.update(fake_dict, batch_size=10000, use_serial=True)

query_obstetricia = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Cl_nica_Obstreticia__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Obstetricia = sf.smart_query(query_obstetricia)
dt_Obstetricia["Clínica"] = "Obstetricia"
dt_Obstetricia["Profesional"] = np.where(
    dt_Obstetricia["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Obstetricia = dt_Obstetricia.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Obstetricia = dt_Obstetricia[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Obstetricia.shape

100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


(0, 8)

Ergonomia

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_de_Ergonomia__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_de_Ergonomia__c.update(fake_dict, batch_size=10000, use_serial=True)

query_ergonomia = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_de_Ergonomia__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Ergonomia = sf.smart_query(query_ergonomia)
dt_Ergonomia["Clínica"] = "Ergonomia"
dt_Ergonomia["Profesional"] = np.where(
    dt_Ergonomia["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Ergonomia = dt_Ergonomia.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Ergonomia = dt_Ergonomia[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Ergonomia.shape

100%|██████████| 1/1 [00:02<00:00,  2.53s/it]


(17, 8)

Telemedicina

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_Telemedicina__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_Telemedicina__c.update(fake_dict, batch_size=10000, use_serial=True)

query_telemedicina = "SELECT Id,Name,CreatedDate,Canal__c,  Realizado__c, Tipo_de_Atenci_n__c, Owner.Name from Ficha_Cl_nica_Telemedicina__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Telemedicina = sf.smart_query(query_telemedicina)
dt_Telemedicina["Clínica"] = "Telemedicina"
dt_Telemedicina["Profesional"] = np.where(
    dt_Telemedicina["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Telemedicina = dt_Telemedicina.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Telemedicina = dt_Telemedicina[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Telemedicina.shape

100%|██████████| 1/1 [00:08<00:00,  8.77s/it]


(1944, 8)

General

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Clinica_General__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Clinica_General__c.update(fake_dict, batch_size=10000, use_serial=True)

query_general = "SELECT Id,Name,CreatedDate,Canal__c,Owner.Name,Programa__c from Ficha_Clinica_General__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_General = sf.smart_query(query_general)
dt_General["Clínica"] = dt_General["Programa__c"]
dt_General["Profesional"] = np.where(
    dt_General["Owner.Name"] == "No - Reply", "App", "Profesional"
)
dt_General["Realizado"] = "SI"
dt_General["Tipo de atención"] = "Complemento a la atención"

dt_General = dt_General.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_General = dt_General[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_General.shape

100%|██████████| 1/1 [00:02<00:00,  2.70s/it]


(469, 8)

Ficha Front

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Orientaci_n_Medica__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Orientaci_n_Medica__c.update(fake_dict, batch_size=10000, use_serial=True)

query_orientacion = "SELECT Id,Name,CreatedDate,Canal__c,Owner.Name,Tipo_de_Atenci_n__c from Ficha_Orientaci_n_Medica__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Orientacion = sf.smart_query(query_orientacion)
dt_Orientacion["Clínica"] = "Orientación Médica"
dt_Orientacion["Profesional"] = np.where(
    dt_Orientacion["Owner.Name"] == "No - Reply", "App", "Profesional"
)
dt_Orientacion["Realizado"] = "SI"
#dt_Front["Tipo de atención"] = "Complemento a la atención"

dt_Orientacion = dt_Orientacion.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Orientacion = dt_Orientacion[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Orientacion.shape

100%|██████████| 1/1 [00:07<00:00,  7.90s/it]


(9, 8)

Orientacion medica

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Front__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Front__c.update(fake_dict, batch_size=10000, use_serial=True)

query_front = "SELECT Id,Name,CreatedDate,Canal__c,Owner.Name,Tipo_de_Atenci_n__c from Ficha_Front__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Front = sf.smart_query(query_front)
dt_Front["Clínica"] = "Ficha Front"
dt_Front["Profesional"] = np.where(
    dt_Front["Owner.Name"] == "No - Reply", "App", "Profesional"
)
dt_Front["Realizado"] = "SI"
#dt_Front["Tipo de atención"] = "Complemento a la atención"

dt_Front = dt_Front.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Front = dt_Front[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Front.shape

100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


(493, 8)

prevencio cancer de mamas

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Prevencion_Cancer_de_Mamas__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Prevencion_Cancer_de_Mamas__c.update(fake_dict, batch_size=10000, use_serial=True)

query_mamas = "SELECT Id,Name,CreatedDate,Canal__c,Owner.Name,Tipo_de_Atenci_n__c from Ficha_Prevencion_Cancer_de_Mamas__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Mamas = sf.smart_query(query_mamas)
dt_Mamas["Clínica"] = "Prevencion Cancer de Mamas"
dt_Mamas["Profesional"] = np.where(
    dt_Mamas["Owner.Name"] == "No - Reply", "App", "Profesional"
)
dt_Mamas["Realizado"] = "SI"
#dt_Mamas["Tipo de atención"] = "Complemento a la atención"

dt_Mamas = dt_Mamas.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
    }
)
dt_Mamas = dt_Mamas[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Mamas.shape

100%|██████████| 1/1 [00:02<00:00,  2.69s/it]


(0, 8)

Ficha de Activación

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Activaciones__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Activaciones__c.update(fake_dict, batch_size=10000, use_serial=True)

query_activacion = "SELECT Id,Name,CreatedDate,Canal__c,Owner.Name,Tipo_de_Atenci_n__c,Area__c from Activaciones__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Activacion = sf.smart_query(query_activacion)
#dt_Activacion["Clínica"] = "Ficha de Activación"
dt_Activacion["Profesional"] = np.where(
    dt_Activacion["Owner.Name"] == "No - Reply", "App", "Profesional"
)
dt_Activacion["Realizado"] = "SI"
#dt_Activacion["Tipo de atención"] = "Complemento a la atención"

dt_Activacion = dt_Activacion.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
        "Area__c": "Clínica",
    }
)
dt_Activacion = dt_Activacion[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        #"Fecha agendamiento",
    ]
]
dt_Activacion.shape

100%|██████████| 1/1 [00:03<00:00,  3.43s/it]


(938, 8)

Psicologia

In [ ]:
vacio = "SELECT Id, EmpresaContactoConvenio__c from Ficha_Cl_nica_Asistencia_Emocional__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024  and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
Cita_Servicio = sf.smart_query(vacio)
Cita_Servicio = Cita_Servicio[(Cita_Servicio["EmpresaContactoConvenio__c"] == "-")]
Cita_Servicio = Cita_Servicio[["Id"]]
fake_dict = Cita_Servicio.to_dict("records")
sf.bulk.Ficha_Cl_nica_Asistencia_Emocional__c.update(fake_dict, batch_size=10000, use_serial=True)

query_psicologia = "SELECT Id,Name,CreatedDate,Canal__c,Tipo_de_Atenci_n__c, Motivo_de_consulta_manifiesto__c,Realizado__c, Owner.Name from Ficha_Cl_nica_Asistencia_Emocional__c WHERE CALENDAR_YEAR(CreatedDate ) >= 2024 and CALENDAR_MONTH(CreatedDate ) >= 4 ORDER BY CreatedDate"
dt_Psicologia = sf.smart_query(query_psicologia)
dt_Psicologia["Clínica"] = "Psicologia"
dt_Psicologia["Profesional"] = np.where(
    dt_Psicologia["Owner.Name"] == "No - Reply", "App", "Profesional"
)

dt_Psicologia["Motivo_de_consulta_manifiesto__c"] = np.where(
    dt_Psicologia["Motivo_de_consulta_manifiesto__c"] == "-",
    "Otros",
    dt_Psicologia["Motivo_de_consulta_manifiesto__c"],
)
#dt_Activacion["Realizado"] = "SI"
#dt_Activacion["Tipo de atención"] = "Complemento a la atención"

dt_Psicologia = dt_Psicologia.rename(
    columns={
        "Name": "Ficha",
        "CreatedDate": "Fecha creación",
        "Canal__c": "Canal",
        "Realizado__c": "Realizado",
        "Tipo_de_Atenci_n__c": "Tipo de atención",
        #"Area__c": "Clínica",
         "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
        
    }
)
dt_Psicologia = dt_Psicologia[
    [
        "Id",
        "Ficha",
        "Fecha creación",
        "Canal",
        "Realizado",
        "Tipo de atención",
        "Profesional",        
        "Clínica",
        "Motivo de consulta"
        #"Fecha agendamiento",
    ]
]
dt_Psicologia.shape

100%|██████████| 1/1 [00:03<00:00,  3.47s/it]


(1067, 9)

In [ ]:
Fichas_Clinicas = pd.concat(
    [
        CitaDeServicioA,
        CitaDeServicioO,
        dt_Psicologia,
        dt_Cronico,
        dt_Deporte,
        dt_Nutricion,
        dt_AppSex,
        dt_Sueno,
        dt_Activacion,
        dt_Mamas,
        dt_Front,
        dt_Orientacion,
        dt_General,
        dt_Telemedicina,
        dt_Ergonomia,
        dt_Obstetricia,
        dt_PrimerosPasos,
        dt_Veterinaria,
        dt_SaludBucal,
        dt_LegalFamilia,
        dt_AsistenciaSocial,
        dt_SaludSexual
        
    ],
    axis=0,
)
Fichas_Clinicas.shape

(13360, 10)

In [ ]:
Fichas_Clinicas.shape

(13360, 10)

In [ ]:
Fichas_Clinicas.head()

,Id,Ficha,Fecha creación,Canal,Realizado,Tipo de atención,Profesional,Clínica,Fecha agendamiento,Motivo de consulta
0,08pRN000000NhHBYA0,SA-62157,1711933783000,Videollamada,Realizado,Agendamiento,App,Consulta Nutricional,1.712405e+12,NaN
1,08pRN000000NhInYAK,SA-62158,1711933881000,Videollamada,Realizado,Agendamiento,App,Consulta Deportiva,1.712408e+12,NaN
2,08pRN000000NhXJYA0,SA-62159,1711938336000,Teléfono,No realizado,Agendamiento,App,Consulta Nutricional,1.711976e+12,NaN
3,08pRN000000NijVYAS,SA-62162,1711972639000,Videollamada,Realizado,Agendamiento,Profesional,Consulta Psicológica,1.712787e+12,NaN
4,08pRN000000Nil7YAC,SA-62163,1711973400000,Videollamada,Cancelado,Agendamiento,App,Veterinaria,1.712581e+12,NaN


OBjeto reportes

In [ ]:
query_reporte = "SELECT Ficha__c, Paciente__c, Empresa__c, Rut_Titular__c, Relacion__c, Cargo__c, Divisi_n__c FROM Reporte__c"
Reportes_SF = sf.smart_query(query_reporte)
Reportes_SF.shape

100%|██████████| 1/1 [00:12<00:00, 12.75s/it]


(64651, 7)

Empresas

In [ ]:
query_empresa = "SELECT Id, Campa_a__c, Holding__c, Identificador__c, Compania__c, P_liza__c, Corredor__c  from Datos_Empresa__c"
Empresas_SF = sf.smart_query(query_empresa)
Empresas_SF.shape

100%|██████████| 1/1 [00:09<00:00,  9.07s/it]


(8659, 7)

Pacientes

In [ ]:
query_pacientes = "SELECT Id,Name, IdentificationId__pc, HealthCloudGA__Gender__pc,Pa_s_del_Paciente__r.Name, HealthCloudGA__Age__pc from Account where IsPersonAccount = true"
Pacientes_SF = sf.smart_query(query_pacientes)
Pacientes_SF.shape

100%|██████████| 1/1 [06:13<00:00, 373.25s/it]


(2045620, 6)

In [ ]:
reportes_general = pd.merge(
    left=Reportes_SF,
    right=Pacientes_SF,
    how="left",
    left_on="Paciente__c",
    right_on="Id",
)

In [ ]:
reportes_general.shape

(64651, 13)

In [ ]:
reportes_general = pd.merge(
    left=reportes_general,
    right=Empresas_SF,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)

In [ ]:
reportes_general.shape

(64651, 20)

In [ ]:
reportes_general = pd.merge(
    left=reportes_general,
    right=Fichas_Clinicas,
    how="left",
    left_on="Ficha__c",
    right_on="Id",
)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_5672\3215250680.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Id_x'} in the result is deprecated and will raise a MergeError in a future version.
  reportes_general = pd.merge(


In [ ]:
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "163820064")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "153298939")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "191710940")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "182470082")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "16277865")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "16277865K")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "HEGJ930928HMNRRL06")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "16007258K")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "86676540")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "202201466")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "105233469")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "130949185")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "255356798")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "34005945")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "185410072")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "34954659")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "166110890")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "197415029")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "254468134")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "27591679K")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "123890211")].index, inplace=True)
reportes_general.drop(reportes_general[(reportes_general["IdentificationId__pc"] == "153298931")].index, inplace=True)


In [ ]:
reportes_general.Id.fillna("NONE", inplace=True)
reportes_general = reportes_general[reportes_general["Id"] == "NONE"]


In [95]:
reportes_general.shape

(36798, 40)

In [ ]:
reportes_general.to_excel("F:/Empresas SF/reportes/derty_yuiop_2.xlsx", index=False)

In [ ]:
reportes_general.shape

In [ ]:
df_deportes.head(50)

In [ ]:
print(pandas.__version__)

In [ ]:
ficha_deporte.head()

In [ ]:
Reortes_general["Dia"] = pd.to_datetime(Reortes_general["Fecha"]).dt.day

Reortes_general["Mes"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month

Reortes_general["semana"] = pd.to_datetime(Reortes_general["Fecha"]).dt.month_name()

Reortes_general["Año"] = pd.to_datetime(Reortes_general["Fecha"]).dt.year


# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime(
    "%d-%m-%Y"
)
general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "Otros": "Prefiero no responder",
}


clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    "Seguimiento de Pacientes Crónicos": "Pacientes Crónicos",
    "Orientación Médica": "Orientación Médica",
    "Asesoría Deportiva": "Deporte",
    "Asistencia Emocional": "Psicología",
    "Asistencia Nutricional": "Nutrición",
    "Clínica APSSex": "Asesoría Psicológica en Salud Sexual",
    "Clínica Del Sueño": "Clínica Del Sueño",
    "Clínica Veterinaria": "Veterinaria",
    "Ingreso Deportivo Seguro": "Deporte",
    "Clínica Mis Primeros Pasos": "Mis Primeros Pasos",
    "Ingreso Deporte Seguro": "Deporte",
    "APSSex": "Asesoría Psicológica en Salud Sexual",
    "Médico Cirujano": "Telemedicina",
    "Kinesióloga": "Deporte",
    "Coordinadora General de Operaciones": "Nutrición",
    "Coordinadora de Equipo de Nutrición y Deporte": "Deporte",
    "Encargado de Equipo Psicología": "Psicología",
    "Psicólogo": "Psicología",
    "Coordinadora de Telemedicina, Enfermería, Maternidad, Dental y Veterinaria": "Telemedicina",
    "Clínica de Orientación LGBT+": "Orientación LGBT+",
    "Clínica MetLife Prueba": "Orientación Médica",
    "Soporte TI": "Orientación Médica",
    "Paciente Crónico": "Pacientes Crónicos",
    "Consulta Deportiva": "Deporte",
    "Consulta Nutricional": "Nutrición",
    "Consulta Psicológica": "Psicología",
}
Reortes_general.replace({"Clínica": clinicas_}, inplace=True)

Clinicas_metlife = {
    "Mis Primeros Pasos": "Pacientes Crónicos",
    "Ingreso Deporte Seguro": "Deporte",
    "Veterinaria": "Orientación Médica",
}

Canal = {
    "Phone": "LLamada",
    "Video": "Videollamada",
    "Entrante - Chat": "Entrante - Chat Online",
    "Videollamda": "Videollamada",
    "Teléfono": "LLamada",
    "Correo": "Correo electrónico",
}
Reortes_general.replace({"Canal": Canal}, inplace=True)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}
Reortes_general.replace({"Canal": Canal}, inplace=True)
Reortes_general["Meses"] = Reortes_general["Mes"]
Reortes_general.replace({"Meses": meses}, inplace=True)
Reortes_general["Fecha_chilena"] = Reortes_general["Fecha_chilena"].dt.tz_localize(None)
Reortes_general.Rut.fillna("0", inplace=True)
Reortes_general.Canal.fillna("Sin Canal", inplace=True)

In [ ]:
Reportes_con_empresas = pd.merge(
    left=Reortes_general,
    right=empresas_unidas,
    how="left",
    left_on="Id Paciente",
    right_on="Id Account Paciente",
)


Reportes_con_empresas.shape

In [ ]:
Reportes_con_empresas["Rut_x"].str.upper()
Reportes_con_empresas["Rut_y"].str.upper()

Reportes_con_empresas.Empleador_x.fillna(
    Reportes_con_empresas["Nombre Empresa"], inplace=True
)

Salida de datos


In [ ]:
reportes = Reportes_con_empresas[
    [
        "Codigo Ficha",
        "Rut_x",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador_x",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Año",
        "Meses",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]


reportes.drop(reportes[(reportes["Rut_x"] == "153298939")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "191710940")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "182470082")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "16277865")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "HEGJ930928HMNRRL06")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "1122345")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "28131394")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "34005945")].index, inplace=True)
reportes.drop(reportes[(reportes["Rut_x"] == "255356798")].index, inplace=True)


Pais_SF = {"a1U8c000005dAtnEAE": "Chile", "a1U8c000005dAtsEAE": "México"}
reportes.replace({"Pais": Pais_SF}, inplace=True)

reportes = reportes.rename(columns={"Rut_x": "Rut", "Empleador_x": "Empleador"})
reportes["Motivo de consulta"].fillna(0, inplace=True)
reportes["Motivo de consulta"] = np.where(
    (reportes["Clínica"] == "Psicología") & (reportes["Motivo de consulta"] == 0),
    "Ingreso a Psicologia",
    reportes["Motivo de consulta"],
)
reportes.Clínica.fillna("Orientación Médica", inplace=True)
reportes = reportes.drop_duplicates(
    ["Clínica", "Empleador", "Nombre Empresa", "Codigo Ficha"], keep="last"
)
reportes.replace({"Sexo": general_Sexo}, inplace=True)

reportes["Rut"] = reportes["Rut"].str.replace("-", "")
reportes.replace({"Sexo": general_Sexo}, inplace=True)

In [ ]:
hoy = datetime.datetime.now()
print(hoy)
reportes.shape

In [ ]:
reportes.to_excel("C:/Users/jmartinez/Documents/reportes2.xlsx", index=False)
reportes.dtypes

In [ ]:
reportes = reportes[(reportes["Fecha_chilena"] <= hoy)]
reportes.shape

In [ ]:
reportes.shape

In [ ]:
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002436"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002438"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002439"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002444"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002446"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002452"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002453"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002454"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002456"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002458"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002459"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340002460"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340003524"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340003525"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340003526"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340003529"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340003535"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340008474"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340008475"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340008476"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340008481"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340008482"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340008484"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340011103"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340011161"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)
reportes["Corredor"] = np.where(
    (reportes["Fecha"] >= "2023-08-01T00:00:00.000+0000")
    & (reportes["Poliza"] == "340011103"),
    "Aon Risk Services (Chile) Corred. De Seg.",
    reportes["Corredor"],
)

In [ ]:
""" Reportes_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes_completos.xlsx", index=False
) """

Programas_pacientes.to_excel("C:/Users/jmartinez/Documents/Empresas.xlsx", index=False)
# reportes.to_excel("C:/Users/jmartinez/Documents/reportes.xlsx", index=False)

NameError: name 'Programas_pacientes' is not defined

In [ ]:
general_todo = reportes[(reportes["Empleador"] == reportes["Nombre Empresa"])]


# general_todo2 = reportes[(reportes["Empleador"] == reportes["Nombre Empresa"]) & (reportes["Poliza"]=="340002436") | (reportes["Poliza"]=="340002446") ]

In [ ]:
# general_todo = reportes[(reportes["Empleador"] == reportes["Nombre Empresa"])]
reportes.to_excel("C:/Users/jmartinez/Documents/reportes.xlsx", index=False)
general_todo.to_excel("C:/Users/jmartinez/Documents/General_Base.xlsx", index=False)

In [ ]:
""" reportes = [
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Antofagasta Minerals S.A.")
] """

In [ ]:
reportes = reportes[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Año",
        "Meses",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]
Bhp2 = reportes[
    (reportes["Clínica"] == "Psicología") & (reportes["Holding"] == "Grupo BHP")
]
Bhp2 = Bhp2[["Clínica", "Motivo de consulta", "Fecha_chilena"]]
Bhp2.to_excel("F:/Reportes SF/Vida Security/BHP_Base2.xlsx", index=False)

reportes["Motivo de consulta"] = ""

Banco Estado


In [ ]:
Banco_Estado = reportes[
    (reportes["Corredor"] == "Bancoestado Corredores De Seguros S.A.")
    & (reportes["Holding"] != "Sin Holding Asociado")
]
Banco_Estado = Banco_Estado[
    (Banco_Estado["Empleador"] == Banco_Estado["Nombre Empresa"])
]

Banco_Estado.to_excel(
    "F:/Reportes SF/Banco Estado Metlife/Banco_Estado_Base.xlsx", index=False
)

ZURICH


In [ ]:
Zurich = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Zurich.to_excel("F:/Reportes SF/Zurich/Zurich_Base.xlsx", index=False)

Amsa = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Antofagasta Minerals S.A.")
]
Amsa.to_excel("F:/Reportes SF/Zurich/Amsa.xlsx", index=False)
resultadoAmsa = sf.query_all(
    """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
amsaIngreso = pd.DataFrame(resultadoAmsa["records"])

amsaIngreso = amsaIngreso[
    [
        "expr0",
        "expr2",
        "expr1",
    ]
]
amsaIngreso = amsaIngreso.rename(
    columns={
        "expr0": "Año",
        "expr1": "Cantidad",
        "expr2": "Mes",
    }
)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

amsaIngreso.replace({"Mes": meses}, inplace=True)

amsaIngreso.to_excel("F:/Reportes SF/Zurich/Cantida_AMSA.xlsx", index=False)


Bechtel = reportes[
    (reportes["Campaña"] == "Zurich")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Holding"] == "Bechtel")
]
Bechtel.to_excel("F:/Reportes SF/Zurich/Bechtel.xlsx", index=False)

VIDA SECURITY


In [ ]:
Vida_security = reportes[
    (reportes["Campaña"] == "Vida Security")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Vida_security = Vida_security[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Meses",
        "Año",
        "Canal",
        "Clínica",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]
Vida_security.to_excel(
    "F:/Reportes SF/Vida Security/Vida_Security_Base.xlsx", index=False
)


Bhp = reportes[
    (reportes["Holding"] == "Grupo BHP")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Bhp = Bhp[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Ocupacion",
        "Cargo",
        "Departamento",
        "Campaña",
        "Holding",
        "Empleador",
        "Poliza",
        "Fecha",
        "Fecha_chilena",
        "Fecha_chilena2",
        "Dia",
        "Mes",
        "Meses",
        "Año",
        "Canal",
        "Clínica",
        "Nombre Empresa",
        "Codigo EMP",
        "Codigo A",
        "Vigente",
        "Corredor",
        "Pais",
    ]
]


Bhp.to_excel("F:/Reportes SF/Vida Security/BHP_Base.xlsx", index=False)


Sqm = reportes[
    (reportes["Holding"] == "Grupo S.Q.M.")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Sqm.to_excel("F:/Reportes SF/Vida Security/SQM_Base.xlsx", index=False)


Vs_mercer = reportes[
    (reportes["Corredor"] == "Mercer") & (reportes["Campaña"] == "Vida Security")
]
Vs_mercer = Vs_mercer[(Vs_mercer["Empleador"] == Vs_mercer["Nombre Empresa"])]
Vs_mercer.to_excel("F:/Reportes SF/Vida Security/Vs_mercer_Base.xlsx", index=False)

Ferronor = reportes[
    (reportes["Holding"] == "Ferronor") & (reportes["Campaña"] == "Vida Security")
]
Ferronor = Ferronor[(Ferronor["Empleador"] == Ferronor["Nombre Empresa"])]
Ferronor.to_excel("F:/Reportes SF/Vida Security/Ferronor_Base.xlsx", index=False)

Itau = reportes[
    (reportes["Holding"] == "Itaú") & (reportes["Campaña"] == "Vida Security")
]
Itau = Itau[(Itau["Empleador"] == Itau["Nombre Empresa"])]
Itau.to_excel("F:/Reportes SF/Vida Security/Itau_Base.xlsx", index=False)

Enami = Vida_security[
    (reportes["Holding"] == "Enami") & (reportes["Campaña"] == "Vida Security")
]
Enami = Enami[(Enami["Empleador"] == Enami["Nombre Empresa"])]
Enami.to_excel("F:/Reportes SF/Vida Security/Enami_Base.xlsx", index=False)

MERCER


In [ ]:
Mercer = reportes[
    (reportes["Campaña"] == "Mercer")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Mercer.to_excel("F:/Reportes SF/Mercer/Mercer_Base.xlsx", index=False)

SURA


In [ ]:
Sura = reportes[
    (reportes["Campaña"] == "Sura")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Sura.to_excel("F:/Reportes SF/Sura/Sura_Base.xlsx", index=False)

Pesco


In [ ]:
Pesco = reportes[
    (reportes["Campaña"] == "Pesco")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Pesco.to_excel("F:/Reportes SF/Pesco/Pesco_Base.xlsx", index=False)

Milan Fajanovic


In [ ]:
Milan = reportes[
    (reportes["Campaña"] == "Milan Fajanovic")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Milan.to_excel("F:/Reportes SF/Milan Fajanovic/Milan_Fajanovic_Base.xlsx", index=False)

Wom


In [ ]:
wom = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
    & (reportes["Empleador"] == "Wom S.A.")
]
wom.to_excel("F:/Reportes SF/Wom/Wom_Base.xlsx", index=False)

In [ ]:
Forus = reportes[
    (reportes["Campaña"] == "Forus")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Forus.to_excel("F:/Reportes SF/Forus/Forus_Base.xlsx", index=False)

In [ ]:
Cruz = reportes[
    (reportes["Campaña"] == "Farmacias Cruz Verde")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Cruz.to_excel("F:/Reportes SF/Cruz Verde/Cruz_Base.xlsx", index=False)
Cruz.shape

METLIFE


In [ ]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]


Metlife.replace({"Clínica": Clinicas_metlife}, inplace=True)

Metlife.to_excel("F:/Reportes SF/MetLife/Metlife_Base.xlsx", index=False)

In [ ]:
Conosur = reportes[
    (reportes["Codigo EMP"] == "EMP5976")
    | (reportes["Codigo EMP"] == "EMP5977")
    | (reportes["Codigo EMP"] == "EMP5978")
    | (reportes["Codigo EMP"] == "EMP5979")
    | (reportes["Codigo EMP"] == "EMP5980")
    | (reportes["Codigo EMP"] == "EMP5981")
    | (reportes["Codigo EMP"] == "EMP5982")
    | (reportes["Codigo EMP"] == "EMP5983")
    | (reportes["Codigo EMP"] == "EMP5984")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP5985")
    | (reportes["Codigo EMP"] == "EMP7431")
    | (reportes["Codigo EMP"] == "EMP2828")
    | (reportes["Corredor"] == "Cono Sur Corredores De Seguros Ltda.")
]
Conosur.replace({"Clínica": Clinicas_metlife}, inplace=True)
Conosur = Conosur[(Conosur["Empleador"] == Conosur["Nombre Empresa"])]
Conosur.to_excel("F:/Reportes SF/MetLife/Conosur_Base.xlsx", index=False)

In [ ]:
Komatsu = reportes[
    (reportes["Codigo EMP"] == "EMP5913")
    | (reportes["Codigo EMP"] == "EMP9600")
    | (reportes["Codigo EMP"] == "EMP9601")
    | (reportes["Codigo EMP"] == "EMP9602")
    | (reportes["Codigo EMP"] == "EMP9603")
    | (reportes["Codigo EMP"] == "EMP9604")
    | (reportes["Codigo EMP"] == "EMP9605")
    | (reportes["Codigo EMP"] == "EMP9606")
    | (reportes["Codigo EMP"] == "EMP9608")
    | (reportes["Codigo EMP"] == "EMP9609")
    | (reportes["Codigo EMP"] == "EMP9610")
    | (reportes["Codigo EMP"] == "EMP9611")
    | (reportes["Codigo EMP"] == "EMP9612")
    | (reportes["Codigo EMP"] == "EMP9613")
    | (reportes["Codigo EMP"] == "EMP9614")
    | (reportes["Codigo EMP"] == "EMP9615")
    | (reportes["Codigo EMP"] == "EMP9616")
    | (reportes["Codigo EMP"] == "EMP9618")
    | (reportes["Codigo EMP"] == "EMP9619")
]
Komatsu.replace({"Clínica": Clinicas_metlife}, inplace=True)
Komatsu = Komatsu[(Komatsu["Empleador"] == Komatsu["Nombre Empresa"])]
Komatsu.to_excel("F:/Reportes SF/MetLife/Komatsu_Base.xlsx", index=False)

In [ ]:
Burgos = reportes[
    (reportes["Codigo EMP"] == "EMP5924")
    | (reportes["Codigo EMP"] == "EMP5925")
    | (reportes["Codigo EMP"] == "EMP5926")
    | (reportes["Codigo EMP"] == "EMP5927")
    | (reportes["Codigo EMP"] == "EMP5928")
    | (reportes["Codigo EMP"] == "EMP5971")
    | (reportes["Codigo EMP"] == "EMP5972")
    | (reportes["Codigo EMP"] == "EMP5973")
    | (reportes["Codigo EMP"] == "EMP5974")
    | (reportes["Codigo EMP"] == "EMP5975")
    | (reportes["Codigo EMP"] == "EMP7259")
    | (reportes["Codigo EMP"] == "EMP6536")
    | (reportes["Codigo EMP"] == "EMP6959")
    | (reportes["Codigo EMP"] == "EMP6388")
    | (reportes["Corredor"] == "Burgos Y Cía. Corredores De Seguros Ltda.")
]

Burgos = Burgos[(Burgos["Empleador"] == Burgos["Nombre Empresa"])]
Burgos.replace({"Clínica": Clinicas_metlife}, inplace=True)
Burgos.to_excel("F:/Reportes SF/MetLife/Burgos_Base.xlsx", index=False)

In [ ]:
Ultramar = reportes[
    (reportes["Codigo EMP"] == "EMP9232")
    | (reportes["Codigo EMP"] == "EMP9233")
    | (reportes["Codigo EMP"] == "EMP9234")
    | (reportes["Codigo EMP"] == "EMP9235")
    | (reportes["Codigo EMP"] == "EMP9236")
    | (reportes["Codigo EMP"] == "EMP9237")
    | (reportes["Codigo EMP"] == "EMP9238")
    | (reportes["Codigo EMP"] == "EMP9239")
    | (reportes["Codigo EMP"] == "EMP9240")
    | (reportes["Codigo EMP"] == "EMP5955")
    | (reportes["Codigo EMP"] == "EMP5958")
    | (reportes["Codigo EMP"] == "EMP5968")
    | (reportes["Codigo EMP"] == "EMP6713")
    | (reportes["Codigo EMP"] == "EMP7066")
    | (reportes["Codigo EMP"] == "EMP7625")
]

Ultramar = Ultramar[(Ultramar["Empleador"] == Ultramar["Nombre Empresa"])]
Ultramar.replace({"Clínica": Clinicas_metlife}, inplace=True)
Ultramar.to_excel("F:/Reportes SF/MetLife/Ultramar_Base.xlsx", index=False)

In [ ]:
Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_sk = pd.read_excel(Antiguo)

reportes2 = pd.concat([reportes, Antiguo_sk])


sK = reportes2[
    (reportes2["Poliza"] == "340010027")
    | (reportes2["Poliza"] == "340010109")
    | (reportes2["Poliza"] == "340013106")
    | (reportes2["Poliza"] == "340013107")
    | (reportes2["Poliza"] == "340013108")
    | (reportes2["Poliza"] == "340013109")
    | (reportes2["Poliza"] == "340013110")
    | (reportes2["Poliza"] == "340013111")
    | (reportes2["Poliza"] == "340013112")
    | (reportes2["Poliza"] == "340013113")
    | (reportes2["Poliza"] == "340013114")
    | (reportes2["Poliza"] == "340013115")
    | (reportes2["Poliza"] == "340013116")
    | (reportes2["Poliza"] == "340013118")
    | (reportes2["Poliza"] == "340013120")
    | (reportes2["Poliza"] == "340013127")
    | (reportes2["Poliza"] == "340013128")
    | (reportes2["Poliza"] == "340013129")
    | (reportes2["Poliza"] == "340013130")
    | (reportes2["Poliza"] == "340013131")
    | (reportes2["Poliza"] == "340013132")
    | (reportes2["Poliza"] == "340013218")
    | (reportes2["Poliza"] == "340013873")
    | (reportes2["Poliza"] == "340014021")
    | (reportes2["Poliza"] == "340016184")
    | (reportes2["Poliza"] == "340016185")
    | (reportes2["Poliza"] == "340018745")
    | (reportes2["Poliza"] == "340018746")
    | (reportes2["Poliza"] == "340018747")
    | (reportes2["Poliza"] == "340019079")
    | (reportes2["Poliza"] == "340020289")
    | (reportes2["Poliza"] == "340020315")
    | (reportes2["Poliza"] == "340022643")
    | (reportes2["Poliza"] == "340022652")
    | (reportes2["Poliza"] == "340022662")
    | (reportes2["Poliza"] == "340022664")
    | (reportes2["Poliza"] == "340022665")
    | (reportes2["Poliza"] == "340022668")
    | (reportes2["Poliza"] == "340023297")
    | (reportes2["Poliza"] == "340023397")
]

sK.Clínica.fillna("Orientación Médica", inplace=True)
sK.replace({"Sexo": general_Sexo}, inplace=True)
sK.replace({"Clínica": Clinicas_metlife}, inplace=True)
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}
Empresas_Benefits = {
    "BancoChile": "Banco De Chile",
    "bancochile": "Banco De Chile",
    "Fundacion De Salud El Teniente": "Fundación De Salud El Teniente",
    "Serv Biene. Fundación Arturo López Pérez": "Serv. Bienestar Fundación Arturo López Pérez",
}

sK.replace({"Nombre Empresa": Empresas_Benefits}, inplace=True)
sK.replace({"Empleador": Empresas_Benefits}, inplace=True)
sK.replace({"Clínica": Lorena_clinicas}, inplace=True)

sK = sK[(sK["Empleador"] == sK["Nombre Empresa"])]
sK.replace({"Clínica": Clinicas_metlife}, inplace=True)
sK.to_excel("F:/Reportes SF/MetLife/sK_Base.xlsx", index=False)

EY


In [ ]:
Ey = reportes[
    (reportes["Campaña"] == "EY")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Ey["Nombre"] = "NoN"
Ey["Motivo de consulta"] = "NoN"

Ey.to_excel("F:/Reportes SF/EY/EY_Base.xlsx", index=False)

In [ ]:
Thb_Seguros = reportes[
    (reportes["Campaña"] == "THB Seguros COVID")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Thb_Seguros.to_excel(
    "F:/Reportes SF/THB Seguros Covid/Thb_Seguros_Base.xlsx", index=False
)

""" ( reportes["Codigo EMP"] == "EMP5924")
| (reportes["Codigo EMP"] == "EMP5925")
| (reportes["Codigo EMP"] == "EMP5926")
| (reportes["Codigo EMP"] == "EMP5927")
| (reportes["Codigo EMP"] == "EMP5928")
| (reportes["Codigo EMP"] == "EMP5971")
| (reportes["Codigo EMP"] == "EMP5972")
| (reportes["Codigo EMP"] == "EMP5973")
| (reportes["Codigo EMP"] == "EMP5974")
| (reportes["Codigo EMP"] == "EMP5975")
| (reportes["Codigo EMP"] == "EMP7259")
| (reportes["Codigo EMP"] == "EMP6536")
| (reportes["Codigo EMP"] == "EMP6959")
| (reportes["Codigo EMP"] == "EMP6388")

    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP5681")
    | (reportes["Codigo EMP"] == "EMP5682")
    | (reportes["Codigo EMP"] == "EMP5684")
    | (reportes["Codigo EMP"] == "EMP6274")
    | (reportes["Codigo EMP"] == "EMP5685")
    | (reportes["Codigo EMP"] == "EMP5686")
    | (reportes["Codigo EMP"] == "EMP5687")
    | (reportes["Codigo EMP"] == "EMP5688")
    | (reportes["Codigo EMP"] == "EMP5689")
    | (reportes["Codigo EMP"] == "EMP5690")
    | (reportes["Codigo EMP"] == "EMP5691")

    | (reportes["Codigo EMP"] == "EMP5692")
    | (reportes["Codigo EMP"] == "EMP5693")
    | (reportes["Codigo EMP"] == "EMP5694")
    | (reportes["Codigo EMP"] == "EMP5695")
    | (reportes["Codigo EMP"] == "EMP5696")
    | (reportes["Codigo EMP"] == "EMP5697")
    | (reportes["Codigo EMP"] == "EMP5698")
    | (reportes["Codigo EMP"] == "EMP5639")
    | (reportes["Codigo EMP"] == "EMP5640")
    | (reportes["Codigo EMP"] == "EMP5641")
    | (reportes["Codigo EMP"] == "EMP5642")
    | (reportes["Codigo EMP"] == "EMP5643")
    | (reportes["Codigo EMP"] == "EMP5644")

    | (reportes["Codigo EMP"] == "EMP5645")
    | (reportes["Codigo EMP"] == "EMP5646")
    | (reportes["Codigo EMP"] == "EMP5647")
    | (reportes["Codigo EMP"] == "EMP5648")
    | (reportes["Codigo EMP"] == "EMP5649")
    | (reportes["Codigo EMP"] == "EMP5650")
    | (reportes["Codigo EMP"] == "EMP5651")
    | (reportes["Codigo EMP"] == "EMP5652")
    | (reportes["Codigo EMP"] == "EMP5653")
    | (reportes["Codigo EMP"] == "EMP5654")
    | (reportes["Codigo EMP"] == "EMP5655")
    | (reportes["Codigo EMP"] == "EMP7526")
    | (reportes["Codigo EMP"] == "EMP6760")

    | (reportes["Codigo EMP"] == "EMP7678")
    | (reportes["Codigo EMP"] == "EMP6888")
    | (reportes["Codigo EMP"] == "EMP6444")
    | (reportes["Codigo EMP"] == "EMP6406")
    | (reportes["Codigo EMP"] == "EMP5616")
    | (reportes["Codigo EMP"] == "EMP9007")
    |"""


In [ ]:
Willis = reportes[(reportes["Corredor"] == "Willis Towers Watson S.A. Corred. De Seg.")]
Willis.replace({"Clínica": Clinicas_metlife}, inplace=True)
Willis = Willis[(Willis["Empleador"] == Willis["Nombre Empresa"])]
Willis.to_excel("F:/Reportes SF/MetLife/Willis_Base.xlsx", index=False)

sKf = reportes[(reportes["Poliza"] == "340018080")]
sKf.replace({"Clínica": Clinicas_metlife}, inplace=True)
sKf = sKf[(sKf["Empleador"] == sKf["Nombre Empresa"])]
sKf.to_excel("F:/Reportes SF/MetLife/Willis_sKf_Base.xlsx", index=False)

In [ ]:
Aon = reportes[
    (reportes["Codigo EMP"] == "EMP5627")
    | (reportes["Codigo EMP"] == "EMP9181")
    | (reportes["Codigo EMP"] == "EMP9183")
    | (reportes["Codigo EMP"] == "EMP9184")
    | (reportes["Codigo EMP"] == "EMP9185")
    | (reportes["Codigo EMP"] == "EMP9186")
    | (reportes["Codigo EMP"] == "EMP9241")
    | (reportes["Codigo EMP"] == "EMP9256")
    | (reportes["Codigo EMP"] == "EMP9187")
    | (reportes["Codigo EMP"] == "EMP9188")
    | (reportes["Codigo EMP"] == "EMP9189")
    | (reportes["Codigo EMP"] == "EMP9230")
    | (reportes["Codigo EMP"] == "EMP9359")
    | (reportes["Codigo EMP"] == "EMP9365")
    | (reportes["Codigo EMP"] == "EMP9366")
    | (reportes["Codigo EMP"] == "EMP9367")
    | (reportes["Codigo EMP"] == "EMP9368")
    | (reportes["Codigo EMP"] == "EMP5890")
    | (reportes["Codigo EMP"] == "EMP5891")
    | (reportes["Codigo EMP"] == "EMP5892")
    | (reportes["Codigo EMP"] == "EMP5893")
    | (reportes["Codigo EMP"] == "EMP5894")
    | (reportes["Codigo EMP"] == "EMP5895")
    | (reportes["Codigo EMP"] == "EMP5896")
    | (reportes["Codigo EMP"] == "EMP5897")
    | (reportes["Codigo EMP"] == "EMP5898")
    | (reportes["Codigo EMP"] == "EMP5899")
    | (reportes["Codigo EMP"] == "EMP5930")
    | (reportes["Codigo EMP"] == "EMP5931")
    | (reportes["Codigo EMP"] == "EMP5932")
    | (reportes["Codigo EMP"] == "EMP5933")
    | (reportes["Codigo EMP"] == "EMP5934")
    | (reportes["Codigo EMP"] == "EMP5935")
    | (reportes["Codigo EMP"] == "EMP5936")
    | (reportes["Codigo EMP"] == "EMP5937")
    | (reportes["Codigo EMP"] == "EMP5939")
    | (reportes["Codigo EMP"] == "EMP5940")
    | (reportes["Codigo EMP"] == "EMP5941")
    | (reportes["Codigo EMP"] == "EMP5942")
    | (reportes["Codigo EMP"] == "EMP5943")
    | (reportes["Codigo EMP"] == "EMP5944")
    | (reportes["Codigo EMP"] == "EMP5945")
    | (reportes["Codigo EMP"] == "EMP5946")
    | (reportes["Codigo EMP"] == "EMP5947")
    | (reportes["Codigo EMP"] == "EMP5949")
    | (reportes["Codigo EMP"] == "EMP9436")
    | (reportes["Codigo EMP"] == "EMP9437")
    | (reportes["Codigo EMP"] == "EMP9861")
    | (reportes["Codigo EMP"] == "EMP9862")
    | (reportes["Codigo EMP"] == "EMP9864")
    | (reportes["Codigo EMP"] == "EMP9865")
    | (reportes["Codigo EMP"] == "EMP9866")
    | (reportes["Codigo EMP"] == "EMP9873")
    | (reportes["Codigo EMP"] == "EMP9874")
    | (reportes["Codigo EMP"] == "EMP9875")
    | (reportes["Codigo EMP"] == "EMP9892")
    | (reportes["Codigo EMP"] == "EMP9893")
    | (reportes["Codigo EMP"] == "EMP9899")
    | (reportes["Codigo EMP"] == "EMP9901")
    | (reportes["Codigo EMP"] == "EMP9902")
    | (reportes["Codigo EMP"] == "EMP9903")
    | (reportes["Codigo EMP"] == "EMP9904")
    | (reportes["Codigo EMP"] == "EMP9905")
    | (reportes["Codigo EMP"] == "EMP9906")
    | (reportes["Codigo EMP"] == "EMP9907")
    | (reportes["Codigo EMP"] == "EMP9908")
    | (reportes["Codigo EMP"] == "EMP9909")
    | (reportes["Codigo EMP"] == "EMP9493")
    | (reportes["Codigo EMP"] == "EMP9494")
    | (reportes["Codigo EMP"] == "EMP9518")
    | (reportes["Codigo EMP"] == "EMP9539")
    | (reportes["Codigo EMP"] == "EMP9540")
    | (reportes["Codigo EMP"] == "EMP9558")
    | (reportes["Codigo EMP"] == "EMP9741")
    | (reportes["Codigo EMP"] == "EMP9742")
    | (reportes["Codigo EMP"] == "EMP9743")
    | (reportes["Codigo EMP"] == "EMP9745")
    | (reportes["Codigo EMP"] == "EMP9747")
    | (reportes["Codigo EMP"] == "EMP9630")
    | (reportes["Codigo EMP"] == "EMP9631")
    | (reportes["Codigo EMP"] == "EMP9632")
    | (reportes["Codigo EMP"] == "EMP9638")
    | (reportes["Codigo EMP"] == "EMP9755")
    | (reportes["Codigo EMP"] == "EMP6532")
    | (reportes["Codigo EMP"] == "EMP6538")
    | (reportes["Codigo EMP"] == "EMP6555")
    | (reportes["Codigo EMP"] == "EMP6556")
    | (reportes["Codigo EMP"] == "EMP6557")
    | (reportes["Codigo EMP"] == "EMP6558")
    | (reportes["Codigo EMP"] == "EMP6560")
    | (reportes["Codigo EMP"] == "EMP6561")
    | (reportes["Codigo EMP"] == "EMP6562")
    | (reportes["Codigo EMP"] == "EMP6563")
    | (reportes["Codigo EMP"] == "EMP6671")
    | (reportes["Codigo EMP"] == "EMP6717")
    | (reportes["Codigo EMP"] == "EMP6720")
    | (reportes["Codigo EMP"] == "EMP6786")
    | (reportes["Codigo EMP"] == "EMP6864")
    | (reportes["Codigo EMP"] == "EMP6865")
    | (reportes["Codigo EMP"] == "EMP6866")
    | (reportes["Codigo EMP"] == "EMP6867")
    | (reportes["Codigo EMP"] == "EMP6868")
    | (reportes["Codigo EMP"] == "EMP6938")
    | (reportes["Codigo EMP"] == "EMP7003")
    | (reportes["Codigo EMP"] == "EMP7004")
    | (reportes["Codigo EMP"] == "EMP7083")
    | (reportes["Codigo EMP"] == "EMP7176")
    | (reportes["Codigo EMP"] == "EMP7177")
    | (reportes["Codigo EMP"] == "EMP7269")
    | (reportes["Codigo EMP"] == "EMP7337")
    | (reportes["Codigo EMP"] == "EMP7396")
    | (reportes["Codigo EMP"] == "EMP7418")
    | (reportes["Codigo EMP"] == "EMP7513")
    | (reportes["Codigo EMP"] == "EMP7646")
    | (reportes["Codigo EMP"] == "EMP7808")
    | (reportes["Codigo EMP"] == "EMP7863")
    | (reportes["Codigo EMP"] == "EMP9183")
    | (reportes["Codigo EMP"] == "EMP9184")
    | (reportes["Codigo EMP"] == "EMP9185")
    | (reportes["Codigo EMP"] == "EMP9186")
    | (reportes["Codigo EMP"] == "EMP9256")
    | (reportes["Codigo EMP"] == "EMP9187")
    | (reportes["Codigo EMP"] == "EMP9188")
    | (reportes["Codigo EMP"] == "EMP9189")
    | (reportes["Codigo EMP"] == "EMP5890")
    | (reportes["Codigo EMP"] == "EMP5891")
    | (reportes["Codigo EMP"] == "EMP5892")
    | (reportes["Codigo EMP"] == "EMP5893")
    | (reportes["Codigo EMP"] == "EMP5894")
    | (reportes["Codigo EMP"] == "EMP5895")
    | (reportes["Codigo EMP"] == "EMP5896")
    | (reportes["Codigo EMP"] == "EMP5897")
    | (reportes["Codigo EMP"] == "EMP5898")
    | (reportes["Codigo EMP"] == "EMP5899")
    | (reportes["Codigo EMP"] == "EMP5930")
    | (reportes["Codigo EMP"] == "EMP5931")
    | (reportes["Codigo EMP"] == "EMP5932")
    | (reportes["Codigo EMP"] == "EMP5933")
    | (reportes["Codigo EMP"] == "EMP5934")
    | (reportes["Codigo EMP"] == "EMP5935")
    | (reportes["Codigo EMP"] == "EMP5936")
    | (reportes["Codigo EMP"] == "EMP5937")
    | (reportes["Codigo EMP"] == "EMP5939")
    | (reportes["Codigo EMP"] == "EMP5940")
    | (reportes["Codigo EMP"] == "EMP5941")
    | (reportes["Codigo EMP"] == "EMP5944")
    | (reportes["Codigo EMP"] == "EMP5947")
    | (reportes["Codigo EMP"] == "EMP5949")
    | (reportes["Codigo EMP"] == "EMP9436")
    | (reportes["Codigo EMP"] == "EMP9863")
    | (reportes["Codigo EMP"] == "EMP9864")
    | (reportes["Codigo EMP"] == "EMP9865")
    | (reportes["Codigo EMP"] == "EMP9866")
    | (reportes["Codigo EMP"] == "EMP9901")
    | (reportes["Codigo EMP"] == "EMP9902")
    | (reportes["Codigo EMP"] == "EMP9903")
    | (reportes["Codigo EMP"] == "EMP9904")
    | (reportes["Codigo EMP"] == "EMP9905")
    | (reportes["Codigo EMP"] == "EMP9906")
    | (reportes["Codigo EMP"] == "EMP9907")
    | (reportes["Codigo EMP"] == "EMP9908")
    | (reportes["Codigo EMP"] == "EMP9909")
    | (reportes["Codigo EMP"] == "EMP9493")
    | (reportes["Codigo EMP"] == "EMP9494")
    | (reportes["Codigo EMP"] == "EMP9539")
    | (reportes["Codigo EMP"] == "EMP9540")
    | (reportes["Codigo EMP"] == "EMP9558")
    | (reportes["Codigo EMP"] == "EMP9741")
    | (reportes["Codigo EMP"] == "EMP9742")
    | (reportes["Codigo EMP"] == "EMP9627")
    | (reportes["Codigo EMP"] == "EMP9630")
    | (reportes["Codigo EMP"] == "EMP9631")
    | (reportes["Codigo EMP"] == "EMP9632")
    | (reportes["Codigo EMP"] == "EMP9638")
    | (reportes["Codigo EMP"] == "EMP6786")
    | (reportes["Codigo EMP"] == "EMP7597")
    | (reportes["Corredor"] == "Aon Risk Services (Chile) Corred. De Seg.")
]
Aon.replace({"Clínica": Clinicas_metlife}, inplace=True)
Aon = Aon[(Aon["Empleador"] == Aon["Nombre Empresa"])]
Aon.to_excel("F:/Reportes SF/MetLife/Aon_Base.xlsx", index=False)

In [ ]:
Aramark = reportes[
    (reportes["Codigo EMP"] == "EMP9244")
    | (reportes["Codigo EMP"] == "EMP9245")
    | (reportes["Codigo EMP"] == "EMP9246")
    | (reportes["Codigo EMP"] == "EMP9247")
    | (reportes["Codigo EMP"] == "EMP9248")
    | (reportes["Codigo EMP"] == "EMP9249")
    | (reportes["Codigo EMP"] == "EMP9250")
    | (reportes["Codigo EMP"] == "EMP9148")
    | (reportes["Codigo EMP"] == "EMP9149")
    | (reportes["Codigo EMP"] == "EMP9150")
    | (reportes["Codigo EMP"] == "EMP9151")
    | (reportes["Codigo EMP"] == "EMP9152")
    | (reportes["Codigo EMP"] == "EMP9153")
    | (reportes["Codigo EMP"] == "EMP9154")
    | (reportes["Codigo EMP"] == "EMP9155")
    | (reportes["Codigo EMP"] == "EMP9156")
    | (reportes["Codigo EMP"] == "EMP9157")
    | (reportes["Codigo EMP"] == "EMP9158")
    | (reportes["Codigo EMP"] == "EMP9159")
    | (reportes["Codigo EMP"] == "EMP9160")
    | (reportes["Codigo EMP"] == "EMP9161")
    | (reportes["Codigo EMP"] == "EMP9162")
    | (reportes["Codigo EMP"] == "EMP9163")
    | (reportes["Codigo EMP"] == "EMP9164")
    | (reportes["Codigo EMP"] == "EMP9165")
    | (reportes["Codigo EMP"] == "EMP9166")
    | (reportes["Codigo EMP"] == "EMP9167")
    | (reportes["Codigo EMP"] == "EMP5806")
    | (reportes["Codigo EMP"] == "EMP5808")
    | (reportes["Codigo EMP"] == "EMP5810")
    | (reportes["Codigo EMP"] == "EMP9266")
    | (reportes["Codigo EMP"] == "EMP9267")
    | (reportes["Codigo EMP"] == "EMP9268")
    | (reportes["Codigo EMP"] == "EMP9269")
    | (reportes["Codigo EMP"] == "EMP9270")
    | (reportes["Codigo EMP"] == "EMP9271")
    # | (reportes["Corredor"] == "Aon Risk Services (Chile) Corred. De Seg.")
]
Aramark.replace({"Clínica": Clinicas_metlife}, inplace=True)
Aramark = Aramark[(Aramark["Empleador"] == Aramark["Nombre Empresa"])]
Aramark.to_excel("F:/Reportes SF/MetLife/Aramark_Base.xlsx", index=False)

In [ ]:
Benefits = reportes[
    (reportes["Codigo EMP"] == "EMP9190")
    | (reportes["Codigo EMP"] == "EMP9191")
    | (reportes["Codigo EMP"] == "EMP9192")
    | (reportes["Codigo EMP"] == "EMP9193")
    | (reportes["Codigo EMP"] == "EMP9194")
    | (reportes["Codigo EMP"] == "EMP9195")
    | (reportes["Codigo EMP"] == "EMP9196")
    | (reportes["Codigo EMP"] == "EMP9197")
    | (reportes["Codigo EMP"] == "EMP9198")
    | (reportes["Codigo EMP"] == "EMP9199")
    | (reportes["Codigo EMP"] == "EMP9200")
    | (reportes["Codigo EMP"] == "EMP9201")
    | (reportes["Codigo EMP"] == "EMP9202")
    | (reportes["Codigo EMP"] == "EMP9203")
    | (reportes["Codigo EMP"] == "EMP9204")
    | (reportes["Codigo EMP"] == "EMP9205")
    | (reportes["Codigo EMP"] == "EMP9326")
    | (reportes["Codigo EMP"] == "EMP9327")
    | (reportes["Codigo EMP"] == "EMP9328")
    | (reportes["Codigo EMP"] == "EMP9340")
    | (reportes["Codigo EMP"] == "EMP9345")
    | (reportes["Codigo EMP"] == "EMP9349")
    | (reportes["Codigo EMP"] == "EMP9324")
    | (reportes["Codigo EMP"] == "EMP5908")
    | (reportes["Codigo EMP"] == "EMP5909")
    | (reportes["Codigo EMP"] == "EMP5912")
    | (reportes["Codigo EMP"] == "EMP5913")
    | (reportes["Codigo EMP"] == "EMP5914")
    | (reportes["Codigo EMP"] == "EMP5915")
    | (reportes["Codigo EMP"] == "EMP5916")
    | (reportes["Codigo EMP"] == "EMP5917")
    | (reportes["Codigo EMP"] == "EMP5919")
    | (reportes["Codigo EMP"] == "EMP5920")
    | (reportes["Codigo EMP"] == "EMP5921")
    | (reportes["Codigo EMP"] == "EMP5922")
    | (reportes["Codigo EMP"] == "EMP5923")
    | (reportes["Codigo EMP"] == "EMP6001")
    | (reportes["Codigo EMP"] == "EMP9414")
    | (reportes["Codigo EMP"] == "EMP9464")
    | (reportes["Codigo EMP"] == "EMP9840")
    | (reportes["Codigo EMP"] == "EMP9852")
    | (reportes["Codigo EMP"] == "EMP9703")
    | (reportes["Codigo EMP"] == "EMP9600")
    | (reportes["Codigo EMP"] == "EMP9601")
    | (reportes["Codigo EMP"] == "EMP9602")
    | (reportes["Codigo EMP"] == "EMP9603")
    | (reportes["Codigo EMP"] == "EMP9604")
    | (reportes["Codigo EMP"] == "EMP9605")
    | (reportes["Codigo EMP"] == "EMP9606")
    | (reportes["Codigo EMP"] == "EMP9608")
    | (reportes["Codigo EMP"] == "EMP9609")
    | (reportes["Codigo EMP"] == "EMP9610")
    | (reportes["Codigo EMP"] == "EMP9611")
    | (reportes["Codigo EMP"] == "EMP9612")
    | (reportes["Codigo EMP"] == "EMP9613")
    | (reportes["Codigo EMP"] == "EMP9614")
    | (reportes["Codigo EMP"] == "EMP9615")
    | (reportes["Codigo EMP"] == "EMP9616")
    | (reportes["Codigo EMP"] == "EMP9617")
    | (reportes["Codigo EMP"] == "EMP9618")
    | (reportes["Codigo EMP"] == "EMP9619")
    | (reportes["Codigo EMP"] == "EMP9784")
    | (reportes["Codigo EMP"] == "EMP9670")
    | (reportes["Codigo EMP"] == "EMP9674")
    | (reportes["Codigo EMP"] == "EMP9802")
    | (reportes["Codigo EMP"] == "EMP6534")
    | (reportes["Codigo EMP"] == "EMP6540")
    | (reportes["Codigo EMP"] == "EMP6564")
    | (reportes["Codigo EMP"] == "EMP6621")
    | (reportes["Codigo EMP"] == "EMP6640")
    | (reportes["Codigo EMP"] == "EMP6643")
    | (reportes["Codigo EMP"] == "EMP6650")
    | (reportes["Codigo EMP"] == "EMP6653")
    | (reportes["Codigo EMP"] == "EMP6661")
    | (reportes["Codigo EMP"] == "EMP6758")
    | (reportes["Codigo EMP"] == "EMP6763")
    | (reportes["Codigo EMP"] == "EMP6772")
    | (reportes["Codigo EMP"] == "EMP6783")
    | (reportes["Codigo EMP"] == "EMP6792")
    | (reportes["Codigo EMP"] == "EMP6829")
    | (reportes["Codigo EMP"] == "EMP6869")
    | (reportes["Codigo EMP"] == "EMP6935")
    | (reportes["Codigo EMP"] == "EMP6940")
    | (reportes["Codigo EMP"] == "EMP7005")
    | (reportes["Codigo EMP"] == "EMP7033")
    | (reportes["Codigo EMP"] == "EMP7072")
    | (reportes["Codigo EMP"] == "EMP7189")
    | (reportes["Codigo EMP"] == "EMP7243")
    | (reportes["Codigo EMP"] == "EMP7247")
    | (reportes["Codigo EMP"] == "EMP7345")
    | (reportes["Codigo EMP"] == "EMP7507")
    | (reportes["Codigo EMP"] == "EMP7508")
    | (reportes["Codigo EMP"] == "EMP7546")
    | (reportes["Codigo EMP"] == "EMP7564")
    | (reportes["Codigo EMP"] == "EMP7567")
    | (reportes["Codigo EMP"] == "EMP7617")
    | (reportes["Codigo EMP"] == "EMP7639")
    | (reportes["Codigo EMP"] == "EMP7663")
    | (reportes["Codigo EMP"] == "EMP7674")
    | (reportes["Codigo EMP"] == "EMP7702")
    | (reportes["Codigo EMP"] == "EMP7739")
    | (reportes["Codigo EMP"] == "EMP7744")
    | (reportes["Codigo EMP"] == "EMP7784")
    | (reportes["Codigo EMP"] == "EMP7785")
    | (reportes["Codigo EMP"] == "EMP9206")
    | (reportes["Codigo EMP"] == "EMP5910")
    | (reportes["Codigo EMP"] == "EMP5911")
    | (reportes["Codigo EMP"] == "EMP5918")
    | (reportes["Codigo EMP"] == "EMP9607")
    | (reportes["Corredor"] == "Benefits Corredores De Seguros SPA")
]

cantidad_gente("Benefits Corredores De Seguros SPA")


Benefits = Benefits[(Benefits["Empleador"] == Benefits["Nombre Empresa"])]

Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_Lorena = pd.read_excel(Antiguo)
Antiguo_Lorena = Antiguo_Lorena[
    (Antiguo_Lorena["Corredor"] == "Benefits Corredores De Seguros Spa")
]
Benefits = pd.concat([Benefits, Antiguo_Lorena])


Antiguo_gente = (
    "F:/Reportes SF/MetLife/usuarios/cantidad_metlife_antiguo_gente_benefits.xlsx"
)
nuevo_gente = (
    "F:/Reportes SF/MetLife/usuarios/Benefits Corredores De Seguros Spa_gente.xlsx"
)
Antiguo_gente_read = pd.read_excel(Antiguo_gente)
nuevo_gente_read = pd.read_excel(nuevo_gente)

Benefits_gente = pd.concat([Antiguo_gente_read, nuevo_gente_read])


Benefits.Clínica.fillna("Orientación Médica", inplace=True)
Benefits.replace({"Sexo": general_Sexo}, inplace=True)
Benefits.replace({"Clínica": Clinicas_metlife}, inplace=True)
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}
Empresas_Benefits = {
    "BancoChile": "Banco De Chile",
    "bancochile": "Banco De Chile",
    "Fundacion De Salud El Teniente": "Fundación De Salud El Teniente",
    "Serv Biene. Fundación Arturo López Pérez": "Serv. Bienestar Fundación Arturo López Pérez",
}
Benefits_gente.replace({"Empleador": Empresas_Benefits}, inplace=True)
Benefits.replace({"Nombre Empresa": Empresas_Benefits}, inplace=True)
Benefits.replace({"Empleador": Empresas_Benefits}, inplace=True)
Benefits.replace({"Clínica": Lorena_clinicas}, inplace=True)
Benefits = Benefits[(Benefits["Empleador"] == Benefits["Nombre Empresa"])]
Benefits.to_excel("F:/Reportes SF/MetLife/Benefits_Base.xlsx", index=False)
Benefits_gente.to_excel(
    "F:/Reportes SF/MetLife/usuarios/gente_Benefits.xlsx", index=False
)

In [ ]:
wom = reportes[
    (reportes["Campaña"] == "MetLife") & (reportes["Empleador"] == "Wom S.A.")
]
wom = wom[(wom["Empleador"] == wom["Nombre Empresa"])]
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}
Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_Lorena = pd.read_excel(Antiguo)
Antiguo_Lorena = Antiguo_Lorena[(Antiguo_Lorena["Empleador"] == "Wom S.A.")]

wom = pd.concat([wom, Antiguo_Lorena])
wom.replace({"Clínica": Lorena_clinicas}, inplace=True)
wom.to_excel("F:/Reportes SF/MetLife/wom_Base.xlsx", index=False)

In [ ]:
psg = reportes[(reportes["Campaña"] == "Grupo PSG")]
psg = psg[(psg["Empleador"] == psg["Nombre Empresa"])]

psg.to_excel("F:/Reportes SF/Grupo PSG/psg_Base.xlsx", index=False)

In [ ]:
Interno = reportes[
    (reportes["Codigo EMP"] == "EMP5999")
    | (reportes["Codigo EMP"] == "EMP9683")
    | (reportes["Codigo EMP"] == "EMP9684")
    | (reportes["Codigo EMP"] == "EMP9685")
    | (reportes["Codigo EMP"] == "EMP9686")
    | (reportes["Codigo EMP"] == "EMP9687")
    | (reportes["Codigo EMP"] == "EMP9688")
    | (reportes["Codigo EMP"] == "EMP9689")
    | (reportes["Codigo EMP"] == "EMP9690")
    | (reportes["Codigo EMP"] == "EMP9691")
    | (reportes["Codigo EMP"] == "EMP7881")
    # | (reportes["Corredor"] == "Benefits Corredores De Seguros SPA")
]
Interno.replace({"Clínica": Clinicas_metlife}, inplace=True)
Interno = Interno[(Interno["Empleador"] == Interno["Nombre Empresa"])]
Interno.to_excel("F:/Reportes SF/MetLife/Interno_Base.xlsx", index=False)

In [ ]:
Lorena = reportes[
    (reportes["Codigo EMP"] == "EMP5786")
    | (reportes["Codigo EMP"] == "EMP9171")
    | (reportes["Codigo EMP"] == "EMP5785")
    | (reportes["Codigo EMP"] == "EMP5787")
    | (reportes["Codigo EMP"] == "EMP5788")
    | (reportes["Codigo EMP"] == "EMP5789")
    | (reportes["Codigo EMP"] == "EMP5790")
    | (reportes["Codigo EMP"] == "EMP5791")
    | (reportes["Codigo EMP"] == "EMP5792")
    | (reportes["Codigo EMP"] == "EMP5793")
    | (reportes["Codigo EMP"] == "EMP5794")
    | (reportes["Codigo EMP"] == "EMP9310")
    | (reportes["Codigo EMP"] == "EMP9441")
    | (reportes["Codigo EMP"] == "EMP9792")
    | (reportes["Codigo EMP"] == "EMP6830")
    | (reportes["Codigo EMP"] == "EMP6975")
    | (reportes["Codigo EMP"] == "EMP7014")
    | (reportes["Codigo EMP"] == "EMP7045")
    | (reportes["Codigo EMP"] == "EMP7060")
    | (reportes["Codigo EMP"] == "EMP7422")
    | (reportes["Corredor"] == "Lorena Contreras Ugarte")
]
cantidad_gente("Lorena Contreras Ugarte")


Lorena = Lorena[(Lorena["Empleador"] == Lorena["Nombre Empresa"])]

Antiguo = "F:/Reportes SF/MetLife/MetLife_Antiguo.xlsx"
Antiguo_Lorena = pd.read_excel(Antiguo)
Antiguo_Lorena = Antiguo_Lorena[
    (Antiguo_Lorena["Corredor"] == "Lorena Contreras Ugarte")
]
Lorena = pd.concat([Lorena, Antiguo_Lorena])

Antiguo_gente = "F:/Reportes SF/MetLife/usuarios/cantidad_metlife_antiguo_gente.xlsx"
nuevo_gente = "F:/Reportes SF/MetLife/usuarios/Lorena Contreras Ugarte_gente.xlsx"
Antiguo_gente_read = pd.read_excel(Antiguo_gente)
nuevo_gente_read = pd.read_excel(nuevo_gente)

Lorena_gente = pd.concat([Antiguo_gente_read, nuevo_gente_read])
Lorena_gente.to_excel("F:/Reportes SF/MetLife/usuarios/gente_Lorena.xlsx", index=False)

Lorena.Clínica.fillna("Orientación Médica", inplace=True)
Lorena.replace({"Sexo": general_Sexo}, inplace=True)
Lorena.replace({"Clínica": Clinicas_metlife}, inplace=True)
Lorena_clinicas = {
    "Nutricion": "Nutrición",
    "Deportivo": "Deporte",
    "Front": "Orientación Médica",
    "": "Orientación Médica",
    "Enfermería": "Pacientes Crónicos",
}

Lorena.replace({"Clínica": Lorena_clinicas}, inplace=True)

Lorena.to_excel("F:/Reportes SF/MetLife/Lorena_Base.xlsx", index=False)

In [ ]:
Antiguo_Lorena.head()

In [ ]:
Security_metlife = reportes[
    (reportes["Codigo EMP"] == "EMP5952")
    | (reportes["Codigo EMP"] == "EMP5957")
    | (reportes["Codigo EMP"] == "EMP5966")
    | (reportes["Codigo EMP"] == "EMP5989")
    | (reportes["Codigo EMP"] == "EMP6833")
    | (reportes["Codigo EMP"] == "EMP6902")
    | (reportes["Codigo EMP"] == "EMP7332")
    | (reportes["Codigo EMP"] == "EMP7661")
    | (reportes["Codigo EMP"] == "EMP9232")
    | (reportes["Codigo EMP"] == "EMP9233")
    | (reportes["Codigo EMP"] == "EMP9234")
    | (reportes["Codigo EMP"] == "EMP9235")
    | (reportes["Codigo EMP"] == "EMP9236")
    | (reportes["Codigo EMP"] == "EMP9237")
    | (reportes["Codigo EMP"] == "EMP9238")
    | (reportes["Codigo EMP"] == "EMP9239")
    | (reportes["Codigo EMP"] == "EMP9240")
    | (reportes["Codigo EMP"] == "EMP9347")
    | (reportes["Codigo EMP"] == "EMP9287")
    | (reportes["Codigo EMP"] == "EMP5950")
    | (reportes["Codigo EMP"] == "EMP5951")
    | (reportes["Codigo EMP"] == "EMP5953")
    | (reportes["Codigo EMP"] == "EMP5955")
    | (reportes["Codigo EMP"] == "EMP5956")
    | (reportes["Codigo EMP"] == "EMP5958")
    | (reportes["Codigo EMP"] == "EMP5959")
    | (reportes["Codigo EMP"] == "EMP5960")
    | (reportes["Codigo EMP"] == "EMP5961")
    | (reportes["Codigo EMP"] == "EMP5962")
    | (reportes["Codigo EMP"] == "EMP5963")
    | (reportes["Codigo EMP"] == "EMP5964")
    | (reportes["Codigo EMP"] == "EMP5965")
    | (reportes["Codigo EMP"] == "EMP5967")
    | (reportes["Codigo EMP"] == "EMP5968")
    | (reportes["Codigo EMP"] == "EMP5969")
    | (reportes["Codigo EMP"] == "EMP5990")
    | (reportes["Codigo EMP"] == "EMP5991")
    | (reportes["Codigo EMP"] == "EMP9440")
    | (reportes["Codigo EMP"] == "EMP9454")
    | (reportes["Codigo EMP"] == "EMP9455")
    | (reportes["Codigo EMP"] == "EMP9456")
    | (reportes["Codigo EMP"] == "EMP9457")
    | (reportes["Codigo EMP"] == "EMP9458")
    | (reportes["Codigo EMP"] == "EMP9459")
    | (reportes["Codigo EMP"] == "EMP9460")
    | (reportes["Codigo EMP"] == "EMP9512")
    | (reportes["Codigo EMP"] == "EMP9553")
    | (reportes["Codigo EMP"] == "EMP9669")
    | (reportes["Codigo EMP"] == "EMP6434")
    | (reportes["Codigo EMP"] == "EMP6713")
    | (reportes["Codigo EMP"] == "EMP6877")
    | (reportes["Codigo EMP"] == "EMP6898")
    | (reportes["Codigo EMP"] == "EMP6901")
    | (reportes["Codigo EMP"] == "EMP6905")
    | (reportes["Codigo EMP"] == "EMP7010")
    | (reportes["Codigo EMP"] == "EMP7066")
    | (reportes["Codigo EMP"] == "EMP7080")
    | (reportes["Codigo EMP"] == "EMP7094")
    | (reportes["Codigo EMP"] == "EMP7150")
    | (reportes["Codigo EMP"] == "EMP7333")
    | (reportes["Codigo EMP"] == "EMP7504")
    | (reportes["Codigo EMP"] == "EMP7505")
    | (reportes["Codigo EMP"] == "EMP7623")
    | (reportes["Codigo EMP"] == "EMP7625")
    | (reportes["Codigo EMP"] == "EMP7651")
    | (reportes["Codigo EMP"] == "EMP7721")
    | (reportes["Codigo EMP"] == "EMP7724")
    | (reportes["Corredor"] == "Corredores De Seguros Security Ltda.")
]

Security_metlife.replace({"Clínica": Clinicas_metlife}, inplace=True)
Security_metlife = Security_metlife[
    (Security_metlife["Empleador"] == Security_metlife["Nombre Empresa"])
]
Security_metlife.to_excel("F:/Reportes SF/MetLife/Security_Base.xlsx", index=False)

In [ ]:
Thb = reportes[
    (reportes["Codigo EMP"] == "EMP9336")
    | (reportes["Codigo EMP"] == "EMP9442")
    | (reportes["Codigo EMP"] == "EMP9746")
    | (reportes["Codigo EMP"] == "EMP9627")
    | (reportes["Codigo EMP"] == "EMP5618")
    | (reportes["Codigo EMP"] == "EMP5619")
    | (reportes["Codigo EMP"] == "EMP5620")
    | (reportes["Codigo EMP"] == "EMP5621")
    | (reportes["Codigo EMP"] == "EMP5622")
    | (reportes["Codigo EMP"] == "EMP5623")
    | (reportes["Codigo EMP"] == "EMP5624")
    | (reportes["Codigo EMP"] == "EMP5625")
    | (reportes["Codigo EMP"] == "EMP5626")
    | (reportes["Codigo EMP"] == "EMP5628")
    | (reportes["Codigo EMP"] == "EMP5629")
    | (reportes["Codigo EMP"] == "EMP5630")
    | (reportes["Codigo EMP"] == "EMP5631")
    | (reportes["Codigo EMP"] == "EMP5632")
    | (reportes["Codigo EMP"] == "EMP5633")
    | (reportes["Codigo EMP"] == "EMP5634")
    | (reportes["Codigo EMP"] == "EMP5635")
    | (reportes["Codigo EMP"] == "EMP5636")
    | (reportes["Codigo EMP"] == "EMP5637")
    | (reportes["Codigo EMP"] == "EMP5638")
    | (reportes["Codigo EMP"] == "EMP5669")
    | (reportes["Codigo EMP"] == "EMP5670")
    | (reportes["Codigo EMP"] == "EMP5671")
    | (reportes["Codigo EMP"] == "EMP5672")
    | (reportes["Codigo EMP"] == "EMP5673")
    | (reportes["Codigo EMP"] == "EMP5674")
    | (reportes["Codigo EMP"] == "EMP5675")
    | (reportes["Codigo EMP"] == "EMP5677")
    | (reportes["Codigo EMP"] == "EMP5721")
    | (reportes["Codigo EMP"] == "EMP5722")
    | (reportes["Codigo EMP"] == "EMP5723")
    | (reportes["Codigo EMP"] == "EMP5724")
    | (reportes["Codigo EMP"] == "EMP5725")
    | (reportes["Codigo EMP"] == "EMP5726")
    | (reportes["Codigo EMP"] == "EMP5727")
    | (reportes["Codigo EMP"] == "EMP5728")
    | (reportes["Codigo EMP"] == "EMP5729")
    | (reportes["Codigo EMP"] == "EMP5730")
    | (reportes["Codigo EMP"] == "EMP5731")
    | (reportes["Codigo EMP"] == "EMP5732")
    | (reportes["Codigo EMP"] == "EMP5733")
    | (reportes["Codigo EMP"] == "EMP5734")
    | (reportes["Codigo EMP"] == "EMP5735")
    | (reportes["Codigo EMP"] == "EMP5736")
    | (reportes["Codigo EMP"] == "EMP5737")
    | (reportes["Codigo EMP"] == "EMP5738")
    | (reportes["Codigo EMP"] == "EMP9242")
    | (reportes["Codigo EMP"] == "EMP9121")
    | (reportes["Codigo EMP"] == "EMP9122")
    | (reportes["Codigo EMP"] == "EMP9123")
    | (reportes["Codigo EMP"] == "EMP9168")
    | (reportes["Codigo EMP"] == "EMP9137")
    | (reportes["Codigo EMP"] == "EMP9320")
    | (reportes["Codigo EMP"] == "EMP9364")
    | (reportes["Codigo EMP"] == "EMP9257")
    | (reportes["Codigo EMP"] == "EMP9258")
    | (reportes["Codigo EMP"] == "EMP9259")
    | (reportes["Codigo EMP"] == "EMP9260")
    | (reportes["Codigo EMP"] == "EMP9261")
    | (reportes["Codigo EMP"] == "EMP9262")
    | (reportes["Codigo EMP"] == "EMP9461")
    | (reportes["Codigo EMP"] == "EMP9463")
    | (reportes["Codigo EMP"] == "EMP9859")
    | (reportes["Codigo EMP"] == "EMP9887")
    | (reportes["Codigo EMP"] == "EMP9888")
    | (reportes["Codigo EMP"] == "EMP9925")
    | (reportes["Codigo EMP"] == "EMP9952")
    | (reportes["Codigo EMP"] == "EMP9953")
    | (reportes["Codigo EMP"] == "EMP9514")
    | (reportes["Codigo EMP"] == "EMP9515")
    | (reportes["Codigo EMP"] == "EMP9516")
    | (reportes["Codigo EMP"] == "EMP9517")
    | (reportes["Codigo EMP"] == "EMP9956")
    | (reportes["Codigo EMP"] == "EMP9957")
    | (reportes["Codigo EMP"] == "EMP9958")
    | (reportes["Codigo EMP"] == "EMP9970")
    | (reportes["Codigo EMP"] == "EMP9559")
    | (reportes["Codigo EMP"] == "EMP9561")
    | (reportes["Codigo EMP"] == "EMP9565")
    | (reportes["Codigo EMP"] == "EMP9567")
    | (reportes["Codigo EMP"] == "EMP9568")
    | (reportes["Codigo EMP"] == "EMP9569")
    | (reportes["Codigo EMP"] == "EMP9570")
    | (reportes["Codigo EMP"] == "EMP9571")
    | (reportes["Codigo EMP"] == "EMP9572")
    | (reportes["Codigo EMP"] == "EMP9573")
    | (reportes["Codigo EMP"] == "EMP9574")
    | (reportes["Codigo EMP"] == "EMP9576")
    | (reportes["Codigo EMP"] == "EMP9698")
    | (reportes["Codigo EMP"] == "EMP9699")
    | (reportes["Codigo EMP"] == "EMP9700")
    | (reportes["Codigo EMP"] == "EMP9577")
    | (reportes["Codigo EMP"] == "EMP9580")
    | (reportes["Codigo EMP"] == "EMP9581")
    | (reportes["Codigo EMP"] == "EMP9583")
    | (reportes["Codigo EMP"] == "EMP9584")
    | (reportes["Codigo EMP"] == "EMP9585")
    | (reportes["Codigo EMP"] == "EMP9586")
    | (reportes["Codigo EMP"] == "EMP9587")
    | (reportes["Codigo EMP"] == "EMP9588")
    | (reportes["Codigo EMP"] == "EMP9594")
    | (reportes["Codigo EMP"] == "EMP9624")
    | (reportes["Codigo EMP"] == "EMP9625")
    | (reportes["Codigo EMP"] == "EMP9626")
    | (reportes["Codigo EMP"] == "EMP9628")
    | (reportes["Codigo EMP"] == "EMP9629")
    | (reportes["Codigo EMP"] == "EMP9633")
    | (reportes["Codigo EMP"] == "EMP9634")
    | (reportes["Codigo EMP"] == "EMP9635")
    | (reportes["Codigo EMP"] == "EMP9645")
    | (reportes["Codigo EMP"] == "EMP9791")
    | (reportes["Codigo EMP"] == "EMP9799")
    | (reportes["Codigo EMP"] == "EMP9806")
    | (reportes["Codigo EMP"] == "EMP9807")
    | (reportes["Codigo EMP"] == "EMP9691")
    | (reportes["Codigo EMP"] == "EMP6373")
    | (reportes["Codigo EMP"] == "EMP6397")
    | (reportes["Codigo EMP"] == "EMP6410")
    | (reportes["Codigo EMP"] == "EMP6481")
    | (reportes["Codigo EMP"] == "EMP6528")
    | (reportes["Codigo EMP"] == "EMP6612")
    | (reportes["Codigo EMP"] == "EMP6613")
    | (reportes["Codigo EMP"] == "EMP6753")
    | (reportes["Codigo EMP"] == "EMP6875")
    | (reportes["Codigo EMP"] == "EMP6891")
    | (reportes["Codigo EMP"] == "EMP7011")
    | (reportes["Codigo EMP"] == "EMP7012")
    | (reportes["Codigo EMP"] == "EMP7039")
    | (reportes["Codigo EMP"] == "EMP7040")
    | (reportes["Codigo EMP"] == "EMP7041")
    | (reportes["Codigo EMP"] == "EMP7051")
    | (reportes["Codigo EMP"] == "EMP7143")
    | (reportes["Codigo EMP"] == "EMP7170")
    | (reportes["Codigo EMP"] == "EMP7339")
    | (reportes["Codigo EMP"] == "EMP7420")
    | (reportes["Codigo EMP"] == "EMP7568")
    | (reportes["Codigo EMP"] == "EMP7626")
    | (reportes["Codigo EMP"] == "EMP7627")
    | (reportes["Codigo EMP"] == "EMP7633")
    | (reportes["Codigo EMP"] == "EMP7701")
    | (reportes["Codigo EMP"] == "EMP7740")
    | (reportes["Codigo EMP"] == "EMP7746")
    | (reportes["Codigo EMP"] == "EMP7772")
    | (reportes["Codigo EMP"] == "EMP7776")
    | (reportes["Corredor"] == "THB Chile Corredores De Seguros S.A.")
]
Thb.replace({"Clínica": Clinicas_metlife}, inplace=True)
Thb = Thb[(Thb["Empleador"] == Thb["Nombre Empresa"])]
Thb.to_excel("F:/Reportes SF/MetLife/Thb_Base.xlsx", index=False)

In [ ]:
Veritas = reportes[
    (reportes["Codigo EMP"] == "EMP5717")
    | (reportes["Codigo EMP"] == "EMP5718")
    | (reportes["Codigo EMP"] == "EMP5719")
    | (reportes["Codigo EMP"] == "EMP5720")
    | (reportes["Codigo EMP"] == "EMP9959")
    | (reportes["Codigo EMP"] == "EMP9707")
    | (reportes["Codigo EMP"] == "EMP9823")
    | (reportes["Codigo EMP"] == "EMP9824")
    | (reportes["Codigo EMP"] == "EMP6615")
    | (reportes["Codigo EMP"] == "EMP6662")
    | (reportes["Codigo EMP"] == "EMP6778")
    | (reportes["Codigo EMP"] == "EMP6872")
    | (reportes["Codigo EMP"] == "EMP7042")
    | (reportes["Codigo EMP"] == "EMP7052")
    | (reportes["Corredor"] == "Veritas Corredores De Seguros S.A.")
]
Veritas.replace({"Clínica": Clinicas_metlife}, inplace=True)
Veritas = Veritas[(Veritas["Empleador"] == Veritas["Nombre Empresa"])]
Veritas.to_excel("F:/Reportes SF/MetLife/Veritas_Base.xlsx", index=False)

In [ ]:
Mercer = reportes[
    (reportes["Codigo EMP"] == "EMP9162")
    | (reportes["Codigo EMP"] == "EMP5741")
    | (reportes["Codigo EMP"] == "EMP5744")
    | (reportes["Codigo EMP"] == "EMP5755")
    | (reportes["Codigo EMP"] == "EMP5763")
    | (reportes["Codigo EMP"] == "EMP5772")
    | (reportes["Codigo EMP"] == "EMP5795")
    | (reportes["Codigo EMP"] == "EMP5811")
    | (reportes["Codigo EMP"] == "EMP3852")
    | (reportes["Codigo EMP"] == "EMP9377")
    | (reportes["Codigo EMP"] == "EMP9544")
    | (reportes["Codigo EMP"] == "EMP9714")
    | (reportes["Codigo EMP"] == "EMP9715")
    | (reportes["Codigo EMP"] == "EMP9716")
    | (reportes["Codigo EMP"] == "EMP9717")
    | (reportes["Codigo EMP"] == "EMP9721")
    | (reportes["Codigo EMP"] == "EMP9722")
    | (reportes["Codigo EMP"] == "EMP9723")
    | (reportes["Codigo EMP"] == "EMP6464")
    | (reportes["Codigo EMP"] == "EMP7653")
    | (reportes["Codigo EMP"] == "EMP9244")
    | (reportes["Codigo EMP"] == "EMP9245")
    | (reportes["Codigo EMP"] == "EMP9246")
    | (reportes["Codigo EMP"] == "EMP9247")
    | (reportes["Codigo EMP"] == "EMP9248")
    | (reportes["Codigo EMP"] == "EMP9249")
    | (reportes["Codigo EMP"] == "EMP9250")
    | (reportes["Codigo EMP"] == "EMP9140")
    | (reportes["Codigo EMP"] == "EMP9141")
    | (reportes["Codigo EMP"] == "EMP9148")
    | (reportes["Codigo EMP"] == "EMP9149")
    | (reportes["Codigo EMP"] == "EMP9150")
    | (reportes["Codigo EMP"] == "EMP9151")
    | (reportes["Codigo EMP"] == "EMP9152")
    | (reportes["Codigo EMP"] == "EMP9153")
    | (reportes["Codigo EMP"] == "EMP9154")
    | (reportes["Codigo EMP"] == "EMP9155")
    | (reportes["Codigo EMP"] == "EMP9156")
    | (reportes["Codigo EMP"] == "EMP9157")
    | (reportes["Codigo EMP"] == "EMP9158")
    | (reportes["Codigo EMP"] == "EMP9159")
    | (reportes["Codigo EMP"] == "EMP9160")
    | (reportes["Codigo EMP"] == "EMP9161")
    | (reportes["Codigo EMP"] == "EMP9163")
    | (reportes["Codigo EMP"] == "EMP9164")
    | (reportes["Codigo EMP"] == "EMP9165")
    | (reportes["Codigo EMP"] == "EMP9166")
    | (reportes["Codigo EMP"] == "EMP9167")
    | (reportes["Codigo EMP"] == "EMP9127")
    | (reportes["Codigo EMP"] == "EMP9130")
    | (reportes["Codigo EMP"] == "EMP9131")
    | (reportes["Codigo EMP"] == "EMP5742")
    | (reportes["Codigo EMP"] == "EMP5743")
    | (reportes["Codigo EMP"] == "EMP5745")
    | (reportes["Codigo EMP"] == "EMP5746")
    | (reportes["Codigo EMP"] == "EMP5747")
    | (reportes["Codigo EMP"] == "EMP5748")
    | (reportes["Codigo EMP"] == "EMP5749")
    | (reportes["Codigo EMP"] == "EMP5750")
    | (reportes["Codigo EMP"] == "EMP5751")
    | (reportes["Codigo EMP"] == "EMP5752")
    | (reportes["Codigo EMP"] == "EMP5753")
    | (reportes["Codigo EMP"] == "EMP5754")
    | (reportes["Codigo EMP"] == "EMP5756")
    | (reportes["Codigo EMP"] == "EMP5757")
    | (reportes["Codigo EMP"] == "EMP5758")
    | (reportes["Codigo EMP"] == "EMP5759")
    | (reportes["Codigo EMP"] == "EMP5760")
    | (reportes["Codigo EMP"] == "EMP5761")
    | (reportes["Codigo EMP"] == "EMP5762")
    | (reportes["Codigo EMP"] == "EMP5764")
    | (reportes["Codigo EMP"] == "EMP5765")
    | (reportes["Codigo EMP"] == "EMP5766")
    | (reportes["Codigo EMP"] == "EMP5767")
    | (reportes["Codigo EMP"] == "EMP5768")
    | (reportes["Codigo EMP"] == "EMP5769")
    | (reportes["Codigo EMP"] == "EMP5770")
    | (reportes["Codigo EMP"] == "EMP5771")
    | (reportes["Codigo EMP"] == "EMP5773")
    | (reportes["Codigo EMP"] == "EMP5774")
    | (reportes["Codigo EMP"] == "EMP9276")
    | (reportes["Codigo EMP"] == "EMP9362")
    | (reportes["Codigo EMP"] == "EMP5796")
    | (reportes["Codigo EMP"] == "EMP5797")
    | (reportes["Codigo EMP"] == "EMP5798")
    | (reportes["Codigo EMP"] == "EMP5799")
    | (reportes["Codigo EMP"] == "EMP5800")
    | (reportes["Codigo EMP"] == "EMP5801")
    | (reportes["Codigo EMP"] == "EMP5802")
    | (reportes["Codigo EMP"] == "EMP5803")
    | (reportes["Codigo EMP"] == "EMP5804")
    | (reportes["Codigo EMP"] == "EMP5805")
    | (reportes["Codigo EMP"] == "EMP5806")
    | (reportes["Codigo EMP"] == "EMP5807")
    | (reportes["Codigo EMP"] == "EMP5808")
    | (reportes["Codigo EMP"] == "EMP5809")
    | (reportes["Codigo EMP"] == "EMP5810")
    | (reportes["Codigo EMP"] == "EMP5812")
    | (reportes["Codigo EMP"] == "EMP9264")
    | (reportes["Codigo EMP"] == "EMP9265")
    | (reportes["Codigo EMP"] == "EMP9266")
    | (reportes["Codigo EMP"] == "EMP9267")
    | (reportes["Codigo EMP"] == "EMP9268")
    | (reportes["Codigo EMP"] == "EMP9269")
    | (reportes["Codigo EMP"] == "EMP9270")
    | (reportes["Codigo EMP"] == "EMP9271")
    | (reportes["Codigo EMP"] == "EMP9273")
    | (reportes["Codigo EMP"] == "EMP9274")
    | (reportes["Codigo EMP"] == "EMP9275")
    | (reportes["Codigo EMP"] == "EMP9263")
    | (reportes["Codigo EMP"] == "EMP6178")
    | (reportes["Codigo EMP"] == "EMP6179")
    | (reportes["Codigo EMP"] == "EMP6180")
    | (reportes["Codigo EMP"] == "EMP9445")
    | (reportes["Codigo EMP"] == "EMP9446")
    | (reportes["Codigo EMP"] == "EMP9448")
    | (reportes["Codigo EMP"] == "EMP9449")
    | (reportes["Codigo EMP"] == "EMP9450")
    | (reportes["Codigo EMP"] == "EMP9451")
    | (reportes["Codigo EMP"] == "EMP9462")
    | (reportes["Codigo EMP"] == "EMP9480")
    | (reportes["Codigo EMP"] == "EMP9867")
    | (reportes["Codigo EMP"] == "EMP9868")
    | (reportes["Codigo EMP"] == "EMP9869")
    | (reportes["Codigo EMP"] == "EMP9932")
    | (reportes["Codigo EMP"] == "EMP9933")
    | (reportes["Codigo EMP"] == "EMP9943")
    | (reportes["Codigo EMP"] == "EMP9944")
    | (reportes["Codigo EMP"] == "EMP9945")
    | (reportes["Codigo EMP"] == "EMP9964")
    | (reportes["Codigo EMP"] == "EMP9965")
    | (reportes["Codigo EMP"] == "EMP9596")
    | (reportes["Codigo EMP"] == "EMP9713")
    | (reportes["Codigo EMP"] == "EMP9724")
    | (reportes["Codigo EMP"] == "EMP9725")
    | (reportes["Codigo EMP"] == "EMP9726")
    | (reportes["Codigo EMP"] == "EMP9727")
    | (reportes["Codigo EMP"] == "EMP9728")
    | (reportes["Codigo EMP"] == "EMP9729")
    | (reportes["Codigo EMP"] == "EMP9730")
    | (reportes["Codigo EMP"] == "EMP9731")
    | (reportes["Codigo EMP"] == "EMP9732")
    | (reportes["Codigo EMP"] == "EMP9733")
    | (reportes["Codigo EMP"] == "EMP9738")
    | (reportes["Codigo EMP"] == "EMP9636")
    | (reportes["Codigo EMP"] == "EMP9640")
    | (reportes["Codigo EMP"] == "EMP9641")
    | (reportes["Codigo EMP"] == "EMP9642")
    | (reportes["Codigo EMP"] == "EMP9643")
    | (reportes["Codigo EMP"] == "EMP9644")
    | (reportes["Codigo EMP"] == "EMP6366")
    | (reportes["Codigo EMP"] == "EMP6425")
    | (reportes["Codigo EMP"] == "EMP6433")
    | (reportes["Codigo EMP"] == "EMP6445")
    | (reportes["Codigo EMP"] == "EMP6446")
    | (reportes["Codigo EMP"] == "EMP6450")
    | (reportes["Codigo EMP"] == "EMP6547")
    | (reportes["Codigo EMP"] == "EMP6625")
    | (reportes["Codigo EMP"] == "EMP6626")
    | (reportes["Codigo EMP"] == "EMP6648")
    | (reportes["Codigo EMP"] == "EMP6655")
    | (reportes["Codigo EMP"] == "EMP6656")
    | (reportes["Codigo EMP"] == "EMP6657")
    | (reportes["Codigo EMP"] == "EMP6658")
    | (reportes["Codigo EMP"] == "EMP6659")
    | (reportes["Codigo EMP"] == "EMP6780")
    | (reportes["Codigo EMP"] == "EMP6791")
    | (reportes["Codigo EMP"] == "EMP6897")
    | (reportes["Codigo EMP"] == "EMP6931")
    | (reportes["Codigo EMP"] == "EMP6941")
    | (reportes["Codigo EMP"] == "EMP6942")
    | (reportes["Codigo EMP"] == "EMP6968")
    | (reportes["Codigo EMP"] == "EMP6970")
    | (reportes["Codigo EMP"] == "EMP7074")
    | (reportes["Codigo EMP"] == "EMP7325")
    | (reportes["Codigo EMP"] == "EMP7342")
    | (reportes["Codigo EMP"] == "EMP7415")
    | (reportes["Codigo EMP"] == "EMP7425")
    | (reportes["Codigo EMP"] == "EMP7527")
    | (reportes["Codigo EMP"] == "EMP7529")
    | (reportes["Codigo EMP"] == "EMP7622")
    | (reportes["Codigo EMP"] == "EMP7628")
    | (reportes["Codigo EMP"] == "EMP7644")
    | (reportes["Codigo EMP"] == "EMP7645")
    | (reportes["Codigo EMP"] == "EMP7710")
    | (reportes["Codigo EMP"] == "EMP7711")
    | (reportes["Codigo EMP"] == "EMP7773")
    | (reportes["Codigo EMP"] == "EMP7774")
    | (reportes["Codigo EMP"] == "EMP7775")
    | (reportes["Codigo EMP"] == "EMP7790")
    | (reportes["Codigo EMP"] == "EMP7792")
    | (reportes["Codigo EMP"] == "EMP7874")
    | (reportes["Codigo EMP"] == "EMP7879")
    | (reportes["Corredor"] == "Mercer Corredores De Seguros Ltda.")
]
Mercer.replace({"Clínica": Clinicas_metlife}, inplace=True)
Mercer = Mercer[(Mercer["Empleador"] == Mercer["Nombre Empresa"])]
Mercer.to_excel("F:/Reportes SF/MetLife/Mercer_Base.xlsx", index=False)

In [ ]:
soleO = reportes[(reportes["Campaña"] == "Sole Orienta")]

soleO = soleO[(soleO["Empleador"] == soleO["Nombre Empresa"])]
soleO["Empleador"] = (
    soleO["Empleador"].str.title().replace("Sole Orienta", "Probenefits S.A.")
)
soleO["Nombre Empresa"] = (
    soleO["Nombre Empresa"].str.title().replace("Sole Orienta", "Probenefits S.A.")
)
soleO.to_excel("F:/Reportes SF/Sole Orienta/Sole_Orienta_Base.xlsx", index=False)

In [ ]:
nexo = reportes[(reportes["Campaña"] == "Nexo")]

nexo = nexo[(nexo["Empleador"] == nexo["Nombre Empresa"])]
# nexo["Empleador"] = nexo["Empleador"].str.title().replace("Sole Orienta", "Probenefits S.A.")
# soleO["Nombre Empresa"] = soleO["Nombre Empresa"].str.title().replace("Sole Orienta", "Probenefits S.A.")
nexo.to_excel("F:/Reportes SF/Nexo/Nexo_Base.xlsx", index=False)

In [ ]:
# nuevos = "F:/Reportes SF/Sole Orienta/Sole_Orienta_Based.xlsx"
# soleO = pd.read_excel(nuevos)


soleO.to_excel("F:/Reportes SF/Sole Orienta/Sole_Orienta_Base.xlsx", index=False)

proBenefits = soleO[(soleO["Empleador"] == "Probenefit Corredores De Seguros Ltda.")]
proBenefits.to_excel("F:/Reportes SF/Sole Orienta/proBenefits_Base.xlsx", index=False)

technoServer = soleO[(soleO["Empleador"] == "Fundación TechnoServe Chile")]
technoServer.to_excel("F:/Reportes SF/Sole Orienta/technoServer_Base.xlsx", index=False)

rotteryKrauss = soleO[(soleO["Empleador"] == "Ópticas Rotter y Krauss Ltda.")]
rotteryKrauss.to_excel(
    "F:/Reportes SF/Sole Orienta/rotteryKrauss_Base.xlsx", index=False
)

In [ ]:
Mercado_Libre = reportes[(reportes["Codigo EMP"] == "EMP5617")]

Mercado_Libre = Mercado_Libre[
    (Mercado_Libre["Empleador"] == Mercado_Libre["Nombre Empresa"])
]
Mercado_Libre.to_excel(
    "F:/Reportes SF/Mercado Libre/Mercado_Libre_Base.xlsx", index=False
)

In [ ]:
Ahumada = reportes[(reportes["Codigo EMP"] == "EMP6475")]

Ahumada = Ahumada[(Ahumada["Empleador"] == Ahumada["Nombre Empresa"])]
Ahumada.to_excel("F:/Reportes SF/Farmacias Ahumada/Ahumada_Base.xlsx", index=False)

In [ ]:
Minera_Abra = reportes[
    (reportes["Codigo EMP"] == "EMP7397")
    | (reportes["Codigo EMP"] == "EMP7795")
    | (reportes["Codigo EMP"] == "EMP7796")
    | (reportes["Campaña"] == "Minera El Abra")
]

Minera_Abra = Minera_Abra[(Minera_Abra["Empleador"] == Minera_Abra["Nombre Empresa"])]
Minera_Abra.to_excel("F:/Reportes SF/Minera El Abra/Abra_Base.xlsx", index=False)

In [ ]:
Agrosuper = reportes[(reportes["Holding"] == "Agrosuper")]
Agrosuper.replace({"Clínica": Clinicas_metlife}, inplace=True)
Agrosuper = Agrosuper[(Agrosuper["Empleador"] == Agrosuper["Nombre Empresa"])]
Agrosuper.to_excel("F:/Reportes SF/Metlife/Mercer_Agrosuper_Base.xlsx", index=False)

In [ ]:
Agrosuper2 = reportes[(reportes["Holding"] == "Agrosuper 2")]
Agrosuper2.replace({"Clínica": Clinicas_metlife}, inplace=True)
# Agrosuper2 = Agrosuper2[(Agrosuper2["Empleador"] == Agrosuper2["Nombre Empresa"])]
Agrosuper2.to_excel("F:/Reportes SF/Metlife/Mercer_Agrosuper2_Base.xlsx", index=False)

In [ ]:
Lipigas = reportes[(reportes["Codigo EMP"] == "EMP7310")]

Lipigas = Lipigas[(Lipigas["Empleador"] == Lipigas["Nombre Empresa"])]
Lipigas.to_excel("F:/Reportes SF/Lipigas/Lipigas_Base.xlsx", index=False)

In [ ]:
Socofar = reportes[
    (reportes["Codigo EMP"] == "EMP3780")
    | (reportes["Codigo EMP"] == "EMP6749")
    | (reportes["Codigo EMP"] == "EMP6750")
    | (reportes["Codigo EMP"] == "EMP6751")
    | (reportes["Codigo EMP"] == "EMP7405")
    | (reportes["Codigo EMP"] == "EMP7406")
    | (reportes["Codigo EMP"] == "EMP7407")
]


Socofar = Socofar[(Socofar["Empleador"] == Socofar["Nombre Empresa"])]
Socofar.to_excel("F:/Reportes SF/Socofar/Socofar_Base.xlsx", index=False)

Socofar1 = Socofar[(Socofar["Mes"] == 1)]
Socofar1.to_excel("F:/Reportes SF/Socofar/Socofar_Enero_Base.xlsx", index=False)

Socofar2 = Socofar[(Socofar["Mes"] == 2)]
Socofar2.to_excel("F:/Reportes SF/Socofar/Socofar_Febrero_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 3)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Marzo_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 4)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Abril_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 5)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Mayo_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 6)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Junio_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 7)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Julio_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 8)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Agosto_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 9)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Septiembre_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 10)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Octubre_Base.xlsx", index=False)

Socofar3 = Socofar[(Socofar["Mes"] == 11)]
Socofar3.to_excel("F:/Reportes SF/Socofar/Socofar_Noviembre_Base.xlsx", index=False)

In [ ]:
Kaufmann = reportes[
    (reportes["Codigo EMP"] == "EMP4171")
    | (reportes["Codigo EMP"] == "EMP7308")
    | (reportes["Codigo EMP"] == "EMP7309")
    | (reportes["Campaña"] == "Kaufmann")
]


Kaufmann = Kaufmann[(Kaufmann["Empleador"] == Kaufmann["Nombre Empresa"])]
Kaufmann.to_excel("F:/Reportes SF/Kaufmann/Kaufmann_Base.xlsx", index=False)

In [ ]:
Tressp = reportes[
    (reportes["Codigo EMP"] == "EMP6467")
    # | (reportes["Codigo EMP"] == "EMP6468")
    # | (reportes["Codigo EMP"] == "EMP7607")
]


Tressp = Tressp[(Tressp["Empleador"] == Tressp["Nombre Empresa"])]
Tressp.to_excel("F:/Reportes SF/3SP/3sp_Base.xlsx", index=False)
Tressp.shape

BCI


In [ ]:
Bci = reportes[(reportes["Campaña"] == "BCI Seguros Colectivos")]

Bci = Bci[
    # (Bci["Empleador"] == Bci["Nombre Empresa"]) &
    (Bci["Codigo EMP"] != "EMP6254")
    | (Bci["Codigo EMP"] != "EMP6255")
    | (reportes["Holding"] != "Orbis")
]
Bci = Bci[(Bci["Empleador"] == "BCI Seguros")]

Bci.to_excel("F:/Reportes SF/BCI/BCI_Base.xlsx", index=False)

Cenin = Bci[(reportes["Codigo EMP"] == "EMP6250") | (reportes["Holding"] == "Cemin")]
Cenin = Cenin[(Cenin["Empleador"] == "BCI Seguros")]
Cenin.to_excel("F:/Reportes SF/BCI/Cenin_Base.xlsx", index=False)

Orbis = Bci[
    (reportes["Codigo EMP"] == "EMP6254")
    | (reportes["Codigo EMP"] == "EMP6255")
    | (reportes["Holding"] == "Orbis")
]
Orbis = Orbis[(Orbis["Empleador"] == "BCI Seguros")]
Orbis.to_excel("F:/Reportes SF/BCI/Orbis_Base.xlsx", index=False)

Embonor = Bci[(reportes["Holding"] == "Embonor")]
Embonor = Embonor[(Embonor["Empleador"] == "BCI Seguros")]
Embonor.to_excel("F:/Reportes SF/BCI/Embonor_Base.xlsx", index=False)


Saesa = Bci[(reportes["Holding"] == "Saesa")]
Saesa = Saesa[(Saesa["Empleador"] == "BCI Seguros")]
Saesa.to_excel("F:/Reportes SF/BCI/Saesa_Base.xlsx", index=False)

In [ ]:
bciIndividual = reportes[(reportes["Campaña"] == "BCI Seguros Individuales")]
bciIndividual = bciIndividual[(bciIndividual["Fecha"] > "2023-08-01")]
bciIndividual = bciIndividual[(bciIndividual["Empleador"] == "BCI Seguros")]
bciIndividual.to_excel("F:/Reportes SF/BCI/bciIndividual_Base.xlsx", index=False)

In [ ]:
bciColaborador = reportes[(reportes["Campaña"] == "BCI Seguros Colaboradores")]
# bciColaborador = bciColaborador[(bciColaborador["Fecha"] > "2023-08-01")]
bciColaborador = bciColaborador[(bciColaborador["Empleador"] == "BCI Seguros")]
bciColaborador.to_excel("F:/Reportes SF/BCI/bciColaborador_Base.xlsx", index=False)

In [ ]:
Colmena = reportes[(reportes["Campaña"] == "Colmena")]
clinicas_colmena = {"Mis Primeros Pasos": "Deporte"}
Colmena.replace({"Clínica": clinicas_colmena}, inplace=True)


Colmena = Colmena[(Colmena["Empleador"] == Colmena["Nombre Empresa"])]
Colmena.to_excel("F:/Reportes SF/Colmena/Colmena_Base.xlsx", index=False)

In [ ]:
Pedro = reportes[(reportes["Campaña"] == "Pedro De Valdivia")]


Pedro = Pedro[(Pedro["Empleador"] == Pedro["Nombre Empresa"])]
Pedro.to_excel("F:/Reportes SF/Pedro de Valdivia/Pedro_Base.xlsx", index=False)

In [ ]:
talo_bd = reportes[(reportes["Campaña"] == "Talo")]


talo_bd = talo_bd[(talo_bd["Empleador"] == talo_bd["Nombre Empresa"])]
talo_bd.to_excel("F:/Reportes SF/Talo/Talo_Base.xlsx", index=False)

In [ ]:
Vidacel = reportes[(reportes["Campaña"] == "VidaCel")]


Vidacel = Vidacel[(Vidacel["Empleador"] == Vidacel["Nombre Empresa"])]
Vidacel.to_excel("F:/Reportes SF/Vidacel/Vidacel_Base.xlsx", index=False)

VidacelColab = reportes[(reportes["Campaña"] == "VidaCel Colaboradores")]

VidacelColab = VidacelColab[
    (VidacelColab["Empleador"] == VidacelColab["Nombre Empresa"])
]
VidacelColab.to_excel("F:/Reportes SF/Vidacel/VidacelColab_Base.xlsx", index=False)

In [ ]:
Wfs = reportes[(reportes["Campaña"] == "Holding Wfs Food Services")]

Wfs = Wfs[(Wfs["Empleador"] == Wfs["Nombre Empresa"])]
Wfs.to_excel("F:/Reportes SF/Wfs/Wfs_Base.xlsx", index=False)

In [ ]:
Dermic = reportes[(reportes["Campaña"] == "Dermik")]

Dermic = Dermic[(Dermic["Empleador"] == Dermic["Nombre Empresa"])]
Dermic.to_excel("F:/Reportes SF/Dermik/Dermic_Base.xlsx", index=False)
Dermic.shape

In [ ]:
Instruvalve = reportes[(reportes["Campaña"] == "Instruvalve")]

Instruvalve = Instruvalve[(Instruvalve["Empleador"] == Instruvalve["Nombre Empresa"])]
Instruvalve.to_excel("F:/Reportes SF/Instruvalve/Instruvalve_Base.xlsx", index=False)

In [ ]:
Thb = reportes[(reportes["Campaña"] == "THB Seguros")]

Thb = Thb[(Thb["Empleador"] == Thb["Nombre Empresa"])]
Thb.to_excel("F:/Reportes SF/THB Seguros/Thb_Base.xlsx", index=False)

In [ ]:
Bice = reportes[(reportes["Campaña"] == "BICE Vida")]

Bice = Bice[(Bice["Empleador"] == Bice["Nombre Empresa"])]
Bice.to_excel("F:/Reportes SF/Bice/Bice_Base.xlsx", index=False)

In [ ]:
Lockton = reportes[(reportes["Campaña"] == "Lockton")]

Lockton = Lockton[(Lockton["Empleador"] == Lockton["Nombre Empresa"])]

# Lockton = Lockton[(Lockton["Fecha"] > "2023-09-01") & ]
LocktonA = Lockton[
    (Lockton["Empleador"] == "Albermale Ltda.") & (Lockton["Fecha"] > "2023-09-01")
]
LocktonB = Lockton[(Lockton["Empleador"] != "Albermale Ltda.")]
Lockton = pd.concat([LocktonB, LocktonA], axis=0)

Lockton.to_excel("F:/Reportes SF/Lockton/Lockton_Base.xlsx", index=False)

In [ ]:
Agencia = reportes[(reportes["Campaña"] == "MetLife Agencia")]

Agencia = Agencia[(Agencia["Empleador"] == Agencia["Nombre Empresa"])]
Agencia = Agencia[(Agencia["Fecha_chilena"] >= "2023-09-01 00:01:08.902939")]
Agencia.to_excel("F:/Reportes SF/Agencia/Agencia_Base.xlsx", index=False)

In [ ]:
proChile = reportes[(reportes["Campaña"] == "ProChile")]

proChile = proChile[(proChile["Empleador"] == proChile["Nombre Empresa"])]
# proChile = proChile[(proChile["Fecha_chilena"] >= "2023-05-01 00:01:08.902939")]
proChile = proChile[(proChile["Mes"] == 3) & (proChile["Año"] == 2024)]
proChile.to_excel("F:/Reportes SF/proChile/proChile_Base.xlsx", index=False)

In [ ]:
southbridge = reportes[(reportes["Campaña"] == "Southbridge")]

southbridge = southbridge[(southbridge["Empleador"] == southbridge["Nombre Empresa"])]
# Southbridge = Southbridge[(Southbridge["Fecha_chilena"] >= "2023-05-01 00:01:08.902939")]
southbridge.to_excel("F:/Reportes SF/Southbridge/Southbridge_Base.xlsx", index=False)

southbridgev = reportes[(reportes["Campaña"] == "Southbridge Plan Mujer")]

southbridgev = southbridgev[
    (southbridgev["Empleador"] == southbridgev["Nombre Empresa"])
]
southbridgev = southbridgev[
    (southbridgev["Fecha_chilena"] >= "2024-01-01 00:01:08.902939")
]
southbridgev.to_excel("F:/Reportes SF/Southbridge/SouthbridgeM_Base.xlsx", index=False)

In [ ]:
Construmart = reportes[(reportes["Campaña"] == "Construmart")]
Construmart = Construmart[(Construmart["Fecha"] > "2023-02-01")]

Clinicas_Construmart = {
    "Pacientes Crónicos": "Telemedicina",
    "Clínica Del Sueño": "Telemedicina",
}
Construmart.replace({"Clínica": Clinicas_Construmart}, inplace=True)
Construmart = Construmart[(Construmart["Empleador"] == Construmart["Nombre Empresa"])]
Construmart["Agendamiento"] = "No"
# np.where(Construmart['Codigo Ficha'].str.contains('FSA') == True, 'Si', Construmart['agendamiento'])
Construmart["Agendamiento"] = np.where(
    Construmart["Codigo Ficha"].str.contains("FSA") == True,
    "Si",
    Construmart["Agendamiento"],
)
# Construmart['Codigo Ficha'].star
# Construmart["agendamiento"][Construmart['Codigo Ficha'].str.contains('FSA')]

Construmart.to_excel("F:/Reportes SF/Construmart/Construmart_Base.xlsx", index=False)

In [ ]:
reportesMX = reportes[
    [
        "Codigo Ficha",
        "Rut",
        "Nombre",
        "Sexo",
        "Campaña",
        "Empleador",
        "Fecha",
        "Dia",
        "Mes",
        "Año",
        "Meses",
        "Canal",
        "Clínica",
        "Motivo de consulta",
        "Nombre Empresa",
        "Vigente",
        "Pais",
    ]
]
Mexico = reportesMX[(reportesMX["Pais"] == "México")]
Mexico = Mexico.rename(columns={"Rut": "CURP"})

Mexico


In [ ]:
Mexico = Mexico[(Mexico["Empleador"] == Mexico["Nombre Empresa"])]
Mexico.to_excel("F:/Reportes SF/Mexico/Mexico_Base.xlsx", index=False)

In [ ]:
CareU = Mexico[(Mexico["Campaña"] == "Care U")]

CareU = CareU[(CareU["Empleador"] == CareU["Nombre Empresa"])]
CareU.to_excel("F:/Reportes SF/Mexico/Care U/Care_Base.xlsx", index=False)

In [ ]:
Bbdo = Mexico[(Mexico["Campaña"] == "BBDO")]

Bbdo = Bbdo[(Bbdo["Empleador"] == Bbdo["Nombre Empresa"])]
Bbdo.to_excel("F:/Reportes SF/Mexico/BBDO/BBDO_Base.xlsx", index=False)

In [ ]:
Riddell = Mexico[(Mexico["Campaña"] == "Riddell Sports")]

Riddell = Riddell[(Riddell["Empleador"] == Riddell["Nombre Empresa"])]
Riddell.to_excel(
    "F:/Reportes SF/Mexico/Riddell Sports/Riddell Sports_Base.xlsx", index=False
)

In [ ]:
Cuartel = Mexico[(Mexico["Campaña"] == "Cuartel General")]

Cuartel = Cuartel[(Cuartel["Empleador"] == Cuartel["Nombre Empresa"])]
Cuartel.to_excel("F:/Reportes SF/Mexico/Cuartel General/Cuartel_Base.xlsx", index=False)

In [ ]:
Ducter = Mexico[(Mexico["Campaña"] == "Ducter")]

Ducter = Ducter[(Ducter["Empleador"] == Ducter["Nombre Empresa"])]
Ducter.to_excel("F:/Reportes SF/Mexico/Ducter/Ducter_Base.xlsx", index=False)

In [ ]:
Efectivale = Mexico[(Mexico["Campaña"] == "Efectivale")]

Efectivale = Efectivale[(Efectivale["Empleador"] == Efectivale["Nombre Empresa"])]
Efectivale.to_excel(
    "F:/Reportes SF/Mexico/Efectivale/Efectivale_Base.xlsx", index=False
)

In [ ]:
Ambar = Mexico[(Mexico["Campaña"] == "Ambar")]

Ambar = Ambar[(Ambar["Empleador"] == Ambar["Nombre Empresa"])]
Ambar.to_excel("F:/Reportes SF/Mexico/Ambar/Ambar_Base.xlsx", index=False)

In [ ]:
Charger = Mexico[(Mexico["Campaña"] == "Charger Logistics")]

Charger = Charger[(Charger["Empleador"] == Charger["Nombre Empresa"])]
Charger.to_excel(
    "F:/Reportes SF/Mexico/Charger Logistics/Charger Logistics_Base.xlsx", index=False
)

In [ ]:
Clinica = Mexico[(Mexico["Campaña"] == "Clínica Virtual")]

Clinica = Clinica[(Clinica["Empleador"] == Clinica["Nombre Empresa"])]
Clinica.to_excel(
    "F:/Reportes SF/Mexico/Clinica Virtual/Clinica Virtual_Base.xlsx", index=False
)

In [ ]:
Omg = Mexico[(Mexico["Campaña"] == "OMG")]

Omg = Omg[(Omg["Empleador"] == Omg["Nombre Empresa"])]
Omg.to_excel("F:/Reportes SF/Mexico/OMG/OMG_Base.xlsx", index=False)

In [ ]:
Efisys = Mexico[(Mexico["Campaña"] == "Efisys")]

Efisys = Efisys[(Efisys["Empleador"] == Efisys["Nombre Empresa"])]
Efisys.to_excel("F:/Reportes SF/Mexico/Efisys/Efisys_Base.xlsx", index=False)

In [ ]:
Fadermex = Mexico[(Mexico["Campaña"] == "Fadermex")]

Fadermex = Fadermex[(Fadermex["Empleador"] == Fadermex["Nombre Empresa"])]
Fadermex.to_excel("F:/Reportes SF/Mexico/Fadermex/Fadermex_Base.xlsx", index=False)

In [ ]:
Farmacias = Mexico[(Mexico["Campaña"] == "Farmacias YZA")]

Farmacias = Farmacias[(Farmacias["Empleador"] == Farmacias["Nombre Empresa"])]
Farmacias.to_excel(
    "F:/Reportes SF/Mexico/Farmacias YZA/Farmacias YZA_Base.xlsx", index=False
)

In [ ]:
Geodis = Mexico[(Mexico["Campaña"] == "Geodis")]

Geodis = Geodis[(Geodis["Empleador"] == Geodis["Nombre Empresa"])]
Geodis.to_excel("F:/Reportes SF/Mexico/Geodis/Geodis_Base.xlsx", index=False)

In [ ]:
Hidrosina = Mexico[(Mexico["Campaña"] == "Hidrosina")]

Hidrosina = Hidrosina[(Hidrosina["Empleador"] == Hidrosina["Nombre Empresa"])]
Hidrosina.to_excel("F:/Reportes SF/Mexico/Hidrosina/Hidrosina_Base.xlsx", index=False)

In [ ]:
Icsi = Mexico[(Mexico["Campaña"] == "ICSI")]

Icsi = Icsi[(Icsi["Empleador"] == Icsi["Nombre Empresa"])]
Icsi.to_excel("F:/Reportes SF/Mexico/ICSI/ICSI_Base.xlsx", index=False)

In [ ]:
Infra = Mexico[(Mexico["Campaña"] == "Infra")]

Infra = Infra[(Infra["Empleador"] == Infra["Nombre Empresa"])]
Infra.to_excel("F:/Reportes SF/Mexico/Infra/Infra_Base.xlsx", index=False)


aldo = Mexico[(Mexico["Campaña"] == "Aldo Conti")]

aldo = aldo[(aldo["Empleador"] == aldo["Nombre Empresa"])]
aldo.to_excel("F:/Reportes SF/Mexico/Aldo Conti/Aldo_Base.xlsx", index=False)

In [ ]:
Cursor = Mexico[(Mexico["Campaña"] == "Mi Cursor")]

Cursor = Cursor[(Cursor["Empleador"] == Cursor["Nombre Empresa"])]
Cursor.to_excel("F:/Reportes SF/Mexico/Mi Cursor/Mi Cursor_Base.xlsx", index=False)

In [ ]:
Porter = Mexico[(Mexico["Campaña"] == "Porter Novelli")]

Porter = Porter[(Porter["Empleador"] == Porter["Nombre Empresa"])]
Porter.to_excel(
    "F:/Reportes SF/Mexico/Porter Novelli/Porter Novelli_Base.xlsx", index=False
)

In [ ]:
trans = Mexico[(Mexico["Campaña"] == "Transcontainer")]

trans = trans[(trans["Empleador"] == trans["Nombre Empresa"])]
trans.to_excel(
    "F:/Reportes SF/Mexico/Transcontainer/Transcontainer_Base.xlsx", index=False
)

In [ ]:
Productos = Mexico[(Mexico["Campaña"] == "Productos San Juan")]

Productos = Productos[(Productos["Empleador"] == Productos["Nombre Empresa"])]
Productos.to_excel(
    "F:/Reportes SF/Mexico/Productos San Juan/Productos San Juan_Base.xlsx", index=False
)

In [ ]:
Reckitt = Mexico[(Mexico["Campaña"] == "Reckitt Benckiser")]

Reckitt = Reckitt[(Reckitt["Empleador"] == Reckitt["Nombre Empresa"])]
Reckitt.to_excel(
    "F:/Reportes SF/Mexico/Reckitt Benckiser/Reckitt Benckiser_Base.xlsx", index=False
)

In [ ]:
Sintec = Mexico[(Mexico["Campaña"] == "Sintec")]

Sintec = Sintec[(Sintec["Empleador"] == Sintec["Nombre Empresa"])]
Sintec.to_excel("F:/Reportes SF/Mexico/Sintec/Sintec_Base.xlsx", index=False)

In [ ]:
Ticsa = Mexico[(Mexico["Campaña"] == "Ticsa")]

Ticsa = Ticsa[(Ticsa["Empleador"] == Ticsa["Nombre Empresa"])]
Ticsa.to_excel("F:/Reportes SF/Mexico/Ticsa/Ticsa_Base.xlsx", index=False)

In [ ]:
ddb = Mexico[(Mexico["Campaña"] == "DDB")]

ddb = ddb[(ddb["Empleador"] == ddb["Nombre Empresa"])]
ddb.to_excel("F:/Reportes SF/Mexico/ddb/ddb_Base.xlsx", index=False)

In [ ]:
cyh = Mexico[(Mexico["Campaña"] == "Grupo C&H")]

cyh = cyh[(cyh["Empleador"] == cyh["Nombre Empresa"])]
cyh.to_excel("F:/Reportes SF/Mexico/Grupo C&H/Grupo C&H_Base.xlsx", index=False)

In [ ]:
Metlife = reportes[
    (reportes["Campaña"] == "MetLife")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Metlife.to_excel("C:/Users/jmartinez/Documents/Metlife_derty.xlsx", index=False)

Security = reportes[
    (reportes["Campaña"] == "Vida Security")
    & (reportes["Empleador"] == reportes["Nombre Empresa"])
]
Security.to_excel("C:/Users/jmartinez/Documents/Vida Security.xlsx", index=False)

# Reportes_con_empresas.Empleador_x.fillna(Reportes_con_empresas['Nombre Empresa'], inplace=True)
reportes_francisca = Reportes_con_empresas
reportes_francisca.Empleador_x.fillna("Nones", inplace=True)
reportes_francisca = reportes_francisca[reportes_francisca["Empleador_x"] == "Nones"]
archivo = "SinEmpresas.xlsx"
archivo1 = "Empresas.xlsx"

fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
# archivo_con_fecha = archivo_con_fecha.replace(".", "_")
hasta = Path(
    "E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Pacientes Sin Empresas/"
).joinpath(archivo_con_fecha)
print(hasta)
reportes_francisca.to_excel(hasta, index=False)

In [ ]:
fecha = str(datetime.date.today())
archivo_con_fecha = f"{fecha}_{archivo1}"
archivo_con_fecha = archivo_con_fecha.replace(":", "_")
archivo_con_fecha = archivo_con_fecha.replace("-", "_")
archivo_con_fecha = archivo_con_fecha.replace(" ", "_")
hasta = Path("E:/Siempre/Usuarios/Francisca Rubio Yáñez/TI/Empresas/").joinpath(
    archivo_con_fecha
)
Programas_pacientes.to_excel(hasta, index=False)

hasta = Path("F:/Empresas SF/empresas/").joinpath(archivo_con_fecha)
Programas_pacientes.to_excel(hasta, index=False)